#### <font color='orange'>Imports das Bibliotecas</font>

In [1]:
from datetime import datetime
from random import randint, uniform

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import pickle

from imblearn.under_sampling import RandomUnderSampler
# pip install imbalanced-learn

import warnings
import spacy
from spacy.util import is_package

import plotly.express as px

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)



### <font color='orange'>Introdução</font>

Este projeto tem como objetivo desenvolver uma análise da presença de alunos em aulas e gerar insight para ação da ONG Passos Mágicos. A análise é baseada no histórico de frequência registrado no diário de classe, além de outras informações relevantes sobre as aulas e os estudantes.

Para garantir uma visão abrangente e robusta, foram utilizados dados de bases distintas, incluindo informações sobre o cadastro de alunos, professores, turmas e disciplinas. 


In [12]:
local_arquivos_analise = '../documents/base dados/'


#### <font color='orange'>Carregando as bases</font>

Neste trecho de código, utilizamos a biblioteca `pandas` para realizar a leitura e manipulação dos dados da tabela de frequência diária. O objetivo é transformar a coluna que indica a presença dos alunos (`StPresencaFalta`) em um formato binário, onde **1** representa "presente" e **0** representa "ausente". 

**Passos:**

1. **Leitura do arquivo CSV**: Usamos `pd.read_csv()` para carregar o arquivo de frequência.
2. **Seleção e renomeação das colunas**: Escolhemos as colunas de interesse e renomeamos para facilitar o entendimento.
3. **Transformação dos dados**: Aplicamos uma função condicional (`lambda`) para converter as presenças em valores binários.
4. **Visualização dos dados**: Exibimos uma amostra de 5 linhas para ver como os dados estão formatados.


In [13]:
# Lendo os arquivos CSV usando pandas
df_diarioFreq = pd.read_csv(local_arquivos_analise + "TbDiario/Originais anonimizados/TbDiarioFrequencia.csv")

# Selecionando as colunas e aplicando a lógica condicional
df_diarioFreq = df_diarioFreq[['IdDiarioAula', 'IdAluno', 'StPresencaFalta']].rename(
    columns={
        'IdDiarioAula': 'id_diario_aula',
        'IdAluno': 'id_aluno'
    }
)

# Aplicando a lógica para a coluna de presença
df_diarioFreq['presente'] = df_diarioFreq['StPresencaFalta'].apply(lambda x: 1 if x == 'P' else 0)

# Exibindo uma amostra de 5 linhas
df_diarioFreq.sample(5)

,id_diario_aula,id_aluno,StPresencaFalta,presente
280629,22856,199,P,1
224295,18149,296,J,0
213993,17271,737,P,1
157376,12650,859,P,1
24900,2355,566,P,1


<font color='orange'>**Análise de Presença**</font>

No gráfico abaixo, podemos observar que a porcentagem de alunos presentes é de **78,7%**, enquanto as faltas correspondem a **21,3%**. Esse gráfico de pizza nos dá uma visão clara da distribuição de presenças e faltas em todas as aulas registradas.

In [14]:
# Calculando a porcentagem de presença
total_presencas = df_diarioFreq['presente'].sum()
total_registros = df_diarioFreq['presente'].count()
porcentagem_presenca = (total_presencas / total_registros) * 100
porcentagem_falta = 100 - porcentagem_presenca

# Criando um DataFrame para o gráfico
df_porcentagem = pd.DataFrame({
    'Status': ['Presente', 'Faltou'],
    'Porcentagem': [porcentagem_presenca, porcentagem_falta]
})

# Criando o gráfico de pizza com plotly
fig = px.pie(df_porcentagem, values='Porcentagem', names='Status', title='Porcentagem de Presença')

fig.update_layout(
    title_font=dict(size=20),
    plot_bgcolor='rgba(0, 0, 0, 0)', 
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white')

# Exibindo o gráfico
fig.show()


Neste trecho de código, utilizamos a biblioteca `pandas` para carregar e manipular os dados do diário de aula. O objetivo é organizar as informações e criar variáveis que representem o dia da semana em formato de "dummy variables" (variáveis binárias), facilitando análises futuras.

**Passos:**

1. **Leitura do arquivo CSV**: O arquivo de dados é carregado utilizando `pd.read_csv()`, ignorando possíveis erros de linhas.
2. **Seleção e renomeação de colunas**: Foram escolhidas as colunas necessárias e renomeadas para facilitar a manipulação.
3. **Conversão de data**: A coluna `DataAula` é convertida para o formato `datetime`, e extraímos o dia da semana como um número, onde **0 = Segunda-feira** e **6 = Domingo**.
4. **Criação de variáveis dummy**: Utilizamos `pd.get_dummies()` para criar variáveis binárias que indicam o dia da semana.
5. **Ordenação dos dados**: O DataFrame é ordenado pela data da aula e número da aula, organizando cronologicamente as informações.

Esse processamento prepara os dados para análises mais avançadas baseadas no histórico de aulas.


In [15]:
# Lendo o CSV
df_diarioAula = pd.read_csv(local_arquivos_analise + "TbDiario/Originais anonimizados/TbDiarioAula.csv")

# Selecionando e renomeando colunas
df_diarioAula = df_diarioAula[['IdDiarioAula', 'IdDiario', 'DataAula', 'NumeroAula', 'ConteudoMinistrado', 'IdProfessor']].rename(
    columns={
        'IdDiarioAula': 'id_diario_aula',
        'IdDiario': 'id_diario',
        'DataAula': 'data_aula',
        'NumeroAula': 'num_aula',
        'ConteudoMinistrado': 'conteudo_ministrado',
        'IdProfessor': 'id_professor'
    }
)

# Convertendo a coluna de data para datetime e extraindo o dia da semana
df_diarioAula['data_aula'] = pd.to_datetime(df_diarioAula['data_aula'], format="%Y-%m-%d %H:%M:%S")
df_diarioAula['dia_semana'] = df_diarioAula['data_aula'].dt.weekday

# Criando variáveis dummy para o dia da semana
df_diarioAula = pd.get_dummies(df_diarioAula, columns=['dia_semana'])

# Ordenando pelo campo 'data_aula' e 'num_aula'
df_diarioAula = df_diarioAula.sort_values(by=['data_aula', 'num_aula'])

# Exibindo o DataFrame
df_diarioAula


,id_diario_aula,id_diario,data_aula,num_aula,conteudo_ministrado,id_professor,dia_semana_0,dia_semana_1,dia_semana_2,dia_semana_3,dia_semana_4,dia_semana_5,dia_semana_6
359,376,8,2021-07-01,1,Árvore da bondade.,14,False,False,False,True,False,False,False
363,380,149,2021-07-01,1,Árvore da bondade.,14,False,False,False,True,False,False,False
370,387,161,2021-07-01,1,Árvore da bondade.,14,False,False,False,True,False,False,False
380,397,86,2021-07-01,1,Árvore da bondade.,14,False,False,False,True,False,False,False
388,406,83,2021-07-01,1,Árvore da bondade.,14,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24419,25321,2391,2024-06-28,9,Aplicação da prova substitutiva para alunos co...,2,False,False,False,False,True,False,False
24420,25322,2355,2024-06-28,9,Aplicação da prova substitutiva para alunos co...,2,False,False,False,False,True,False,False
24421,25323,2626,2024-06-28,9,Correção da P2,47,False,False,False,False,True,False,False
24370,25272,2581,2024-06-29,9,Jogos cognitivos,9,False,False,False,False,False,True,False


<font color='orange'>**Análise Presença:**</font>

Podemos verificar outras métricas, como a taxa de presença por dia da semana ou a correlação entre o número de aulas e a taxa de presença. Essas análises podem fornecer insights mais profundos sobre os padrões de frequência e ajudar na identificação de dias com maior ou menor comparecimento.

Por exemplo, poderíamos verificar se há uma diferença significativa na presença dos alunos em determinados dias da semana, o que poderia ajudar a otimizar a distribuição de aulas ou focar em intervenções específicas para reduzir as faltas.


In [16]:
# Juntando as tabelas para adicionar o dia da semana à tabela de frequência
df_completo = pd.merge(df_diarioFreq, df_diarioAula[['id_diario_aula', 'data_aula']], on='id_diario_aula')

# Extraindo o dia da semana e convertendo para nomes dos dias
df_completo['dia_semana'] = df_completo['data_aula'].dt.weekday
dias_semana = {0: 'Segunda', 1: 'Terça', 2: 'Quarta', 3: 'Quinta', 4: 'Sexta', 5: 'Sábado', 6: 'Domingo'}
df_completo['dia_semana'] = df_completo['dia_semana'].map(dias_semana)

# Calculando a taxa de presença por dia da semana
presenca_por_dia = df_completo.groupby('dia_semana')['presente'].mean() * 100

# Ordenando os dias da semana para exibição
presenca_por_dia = presenca_por_dia.reindex(['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado', 'Domingo'])

# Gráfico da taxa de presença por dia da semana
fig_dia_semana = px.bar(presenca_por_dia, 
                        title='Taxa de Presença por Dia da Semana', 
                        labels={'value': 'Porcentagem de Presença', 'index': 'Dia da Semana'},
                        color_discrete_sequence=px.colors.sequential.Blues_r)

fig_dia_semana.update_layout(
    title_font=dict(size=20),
    plot_bgcolor='rgba(0, 0, 0, 0)', 
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white'                
)

# Exibindo o gráfico
fig_dia_semana.show()


A análise de presença por dia da semana mostra uma variação interessante. A maior taxa de presença ocorre no **domingo**, com **85,7%**, enquanto o **sábado** tem a menor taxa de presença, com apenas **50,6%**. Durante os dias úteis, as taxas de presença variam de **75,3%** (sexta-feira) a **81,5%** (terça-feira). Isso sugere que, durante o final de semana, os alunos são menos assíduos, especialmente no sábado, o que pode indicar uma oportunidade de ajustar o planejamento de aulas ou atividades nesse dia para tentar aumentar o engajamento.

In [17]:
# Exibindo o resultado da análise por dia
print(presenca_por_dia)

dia_semana
Segunda    78.877192
Terça      81.505144
Quarta     79.508658
Quinta     76.980671
Sexta      75.337039
Sábado     50.607287
Domingo    85.714286
Name: presente, dtype: float64


<font color='orange'>**Análise de Faltas:**</font>

Criamos a mesma análise para faltas, que será simplesmente o complemento da presença (100% menos a porcentagem de presença para cada dia).


In [18]:
# Calculando a taxa de faltas por dia da semana
faltas_por_dia = 100 - presenca_por_dia

# Gráfico da taxa de faltas por dia da semana 
fig_faltas = px.bar(faltas_por_dia, 
                    title='Taxa de Faltas por Dia da Semana', 
                    labels={'value': 'Porcentagem de Faltas', 'index': 'Dia da Semana'},
                    color_discrete_sequence=px.colors.sequential.Blues_r)

# Ajustando o layout para fundo escuro
fig_faltas.update_layout(
    title_font=dict(size=20),
    plot_bgcolor='rgba(0, 0, 0, 0)', 
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white'                
)

# Exibindo o gráfico
fig_faltas.show()



Assim como na presença, podemos observar que o **sábado** tem a maior taxa de faltas, com **49,4%** de alunos ausentes. Isso contrasta fortemente com o **domingo**, que tem a menor taxa de faltas, com **14,3%**. Durante a semana, as faltas variam entre **18,5%** na terça-feira e **24,7%** na sexta-feira, o que pode sugerir que, no final da semana, a presença diminui consistentemente.


<font color='orange'>**Lendo as tabelas:**</font>

Lendo as tabelas para gerar as principais informações para analises futuras:

- TbDiario;
- TbTurma;
- TbAluno;

In [19]:
# Lendo o arquivo CSV usando pandas
df_diario = pd.read_csv(local_arquivos_analise + "TbDiario/Originais anonimizados/TbDiario.csv")

# Selecionando e renomeando as colunas
df_diario = df_diario[['IdDiario', 'IdTurma', 'IdDisciplina']].rename(
    columns={
        'IdDiario': 'id_diario',
        'IdTurma': 'id_turma',
        'IdDisciplina': 'id_disciplina'
    }
)

# Exibindo uma amostra de 5 linhas
df_diario.sample(5)


,id_diario,id_turma,id_disciplina
244,252,149,16
324,332,181,1
520,572,236,4
326,334,183,4
911,975,585,1


In [20]:

# Lendo o arquivo CSV usando pandas
df_turma = pd.read_csv(local_arquivos_analise + "TbTurma/Originais anonimizados/TbTurma.csv")

# Selecionando e renomeando as colunas
df_turma = df_turma[['IdTurma', 'IdSerie', 'IdPeriodo', 'TurnoPrincipal']].rename(
    columns={
        'IdTurma': 'id_turma',
        'IdSerie': 'id_serie',
        'IdPeriodo': 'id_periodo'
    }
)

# Aplicando a lógica condicional para a coluna 'TurnoPrincipal'
df_turma['turno_turma'] = df_turma['TurnoPrincipal'].apply(lambda x: 
    0 if x == "M" else 
    1 if x == "T" else 
    2 if x == "N" else 
    3 if x == "Z" else 4
)

# Exibindo uma amostra de 5 linhas
df_turma.sample(5)


,id_turma,id_serie,id_periodo,TurnoPrincipal,turno_turma
220,418,2,5,M,0
33,75,8,4,M,0
507,734,15,7,T,1
1,2,2,1,M,0
133,186,15,5,T,1


**Explicação:**

1. **Leitura dos Dados**: O arquivo CSV é carregado com `pd.read_csv()`, ignorando erros de linha.
2. **Seleção e Renomeação de Colunas**: Selecionamos as colunas relevantes e renomeamos para facilitar a manipulação.
3. **Lógica Condicional**: Utilizamos `apply()` com uma função `lambda` para transformar a coluna `TurnoPrincipal` em valores numéricos, conforme especificado:
   - **M** = 0
   - **T** = 1
   - **N** = 2
   - **Z** = 3
   - Qualquer outro valor = 4
4. **Exibição de Amostra**: Mostramos uma amostra de 5 linhas para verificar o resultado.

In [21]:
# Lendo o arquivo CSV usando pandas
df_aluno = pd.read_csv(local_arquivos_analise + "TbAluno/Originais anonimizados/TbAluno.csv")

# Selecionando e renomeando as colunas
df_aluno = df_aluno[['IdAluno', 'DataNascimento', 'Sexo', 'IdPai', 'IdMae', 'CorRaca']].rename(
    columns={
        'IdAluno': 'id_aluno',
        'DataNascimento': 'data_nascimento_aluno'
    }
)

# Convertendo a coluna de data de nascimento para datetime
df_aluno['data_nascimento_aluno'] = pd.to_datetime(df_aluno['data_nascimento_aluno'], format="%Y-%m-%d %H:%M:%S", errors='coerce')

# Calculando a idade do aluno
df_aluno['idade_aluno'] = datetime.now().year - df_aluno['data_nascimento_aluno'].dt.year

# Aplicando a lógica condicional para a coluna 'Sexo'
df_aluno['sexo_aluno'] = df_aluno['Sexo'].apply(lambda x: 0 if x == "M" else 1)

# Verificando se o campo 'IdPai' e 'IdMae' são nulos
df_aluno['tem_pai'] = df_aluno['IdPai'].apply(lambda x: 0 if pd.isnull(x) else 1)
df_aluno['tem_mae'] = df_aluno['IdMae'].apply(lambda x: 0 if pd.isnull(x) else 1)

# Substituindo valores nulos na coluna 'CorRaca' por "None"
df_aluno['raca_aluno'] = df_aluno['CorRaca'].fillna("None")

# Criando variáveis dummy para a coluna 'raca_aluno'
df_aluno = pd.get_dummies(df_aluno, columns=['raca_aluno'])

# Exibindo uma amostra de 5 linhas
df_aluno.sample(5)


,id_aluno,data_nascimento_aluno,Sexo,IdPai,IdMae,CorRaca,idade_aluno,sexo_aluno,tem_pai,tem_mae,raca_aluno_A,raca_aluno_B,raca_aluno_I,raca_aluno_N,raca_aluno_None,raca_aluno_P,raca_aluno_R
1226,1236,2013-04-22 00:00:00,M,NaN,1013,B,11.0,0,0,1,False,True,False,False,False,False,False
261,264,2009-10-04 03:00:00,M,NaN,253,R,15.0,0,0,1,False,False,False,False,False,False,True
37,40,2004-02-02 00:00:00,F,NaN,40,B,20.0,1,0,1,False,True,False,False,False,False,False
258,261,2009-10-12 00:00:00,M,NaN,250,R,15.0,0,0,1,False,False,False,False,False,False,True
301,304,2009-02-28 00:00:00,F,NaN,290,B,15.0,1,0,1,False,True,False,False,False,False,False


**Explicação:**

1. **Leitura dos Dados**: O arquivo CSV é carregado usando `pd.read_csv()`, ignorando erros de linha.
2. **Seleção e Renomeação de Colunas**: Selecionamos as colunas necessárias e as renomeamos.
3. **Conversão de Datas**: A coluna `DataNascimento` é convertida para o formato `datetime` e usamos o ano de nascimento para calcular a idade.
4. **Lógica Condicional**:
   - **Sexo**: `0` para masculino e `1` para feminino.
   - **IdPai e IdMae**: Verificamos se esses campos são nulos (sem pai ou mãe) e atribuímos `0` (não tem) ou `1` (tem).
5. **Preenchimento de Nulos**: Substituímos valores nulos na coluna `CorRaca` por "None".
6. **Variáveis Dummy**: Transformamos a coluna `raca_aluno` em variáveis dummy para análise categórica.
7. **Exibição de Amostra**: Exibimos 5 amostras dos dados para verificação.

In [22]:
df_aluno[['idade_aluno']].describe().T

,count,mean,std,min,25%,50%,75%,max
idade_aluno,2237.0,13.70228,3.656036,1.0,11.0,13.0,16.0,36.0


#### <font color='orange'>Análise Demográfica dos Alunos</font>

**Distribuição de Idade dos Alunos**

Qual a faixa etária predominante dos alunos? Existe uma correlação entre idade e presença nas aulas?


In [12]:
# Criando o histograma da distribuição de idade dos alunos
fig = px.histogram(df_aluno, 
                   x='idade_aluno', 
                   title='Distribuição de Idade dos Alunos', 
                   labels={'idade_aluno': 'Idade dos Alunos'}, 
                   nbins=10,  # Ajuste o número de bins 
                   color_discrete_sequence=['#1f77b4'])

# Ajustando o layout para fundo escuro
fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',  
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white'                
)

# Exibindo o gráfico
fig.show()


<font color='orange'>**Análise do Gráfico de Distribuição de Idade dos Alunos:**</font>

O gráfico mostra a **distribuição etária** dos alunos, concentrando a maioria das idades entre **10 e 19 anos**:

1. **Faixa Etária Predominante**: O pico mais alto ocorre entre **10 e 14 anos**, com aproximadamente **1000 alunos** nessa faixa. Essa é a faixa etária mais representada na amostra.

2. **Queda na Frequência com a Idade**: A partir dos **15 anos**, a quantidade de alunos começa a diminuir, especialmente depois dos **20 anos**, sugerindo que a instituição ou programa pode ter um foco maior em adolescentes.

3. **Alunos mais Velhos**: Embora haja poucos alunos acima dos **20 anos**, ainda existem alunos com até **24 anos**, com uma presença marginal até essa faixa etária.

4. **Possível Foco da ONG**: Dado que a maior parte dos alunos está na faixa de 10 a 19 anos, isso indica que o público atendido pela ONG ou escola provavelmente está em estágios de educação básica e ensino médio.

Esse gráfico sugere que a maior parte do impacto das atividades da ONG será sobre alunos adolescentes. Se há algum esforço para expandir o atendimento a grupos etários mais velhos, uma estratégia específica pode ser necessária para aumentar o engajamento de alunos acima dos 20 anos.

In [29]:

# Juntando as tabelas para trazer as idades dos alunos para a tabela de frequência
df_merged = pd.merge(df_diarioFreq, df_aluno[['id_aluno', 'idade_aluno']], on='id_aluno')

# df_merged = df_merged.drop_duplicates(subset='id_aluno')

# Definindo as faixas etárias
bins = [0, 10, 15, 20, 25, 30, 100]
labels = ['0-10', '11-15', '16-20', '21-25', '26-30', '30+']
df_merged['faixa_etaria'] = pd.cut(df_merged['idade_aluno'], bins=bins, labels=labels, right=False)

# Calculando a porcentagem de presença e faltas por faixa etária
presenca_por_faixa = df_merged.groupby('faixa_etaria')['presente'].mean() * 100
faltas_por_faixa = 100 - presenca_por_faixa

# Exibindo os resultados para análise
print("Porcentagem de Presença por Faixa Etária:")
print(presenca_por_faixa)
print("\nPorcentagem de Faltas por Faixa Etária:")
print(faltas_por_faixa)

# Gráfico de presença por faixa etária (mantido como estava)
fig_presenca = px.bar(presenca_por_faixa, 
                      title='Porcentagem de Presença por Faixa Etária', 
                      labels={'value': 'Porcentagem de Presença', 'faixa_etaria': 'Faixa Etária'},
                      color_discrete_sequence=px.colors.sequential.Blues_r)

fig_presenca.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',  
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white'                
)

fig_presenca.show()

# Gráfico de faltas por faixa etária com cor vermelha clara e rótulo correto
fig_faltas = px.bar(faltas_por_faixa, 
                    title='Porcentagem de Faltas por Faixa Etária', 
                    labels={'value': 'Porcentagem de Faltas', 'faixa_etaria': 'Faixa Etária'},  # Corrigindo o rótulo 'value'
                    color_discrete_sequence=['#FF7F7F'])  # Vermelho claro

fig_faltas.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',  
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white'                
)

fig_faltas.show()


Porcentagem de Presença por Faixa Etária:
faixa_etaria
0-10     85.242775
11-15    81.819095
16-20    74.887585
21-25    60.425920
26-30    73.333333
30+      65.957447
Name: presente, dtype: float64

Porcentagem de Faltas por Faixa Etária:
faixa_etaria
0-10     14.757225
11-15    18.180905
16-20    25.112415
21-25    39.574080
26-30    26.666667
30+      34.042553
Name: presente, dtype: float64


### <font color='orange'>Análise de Correlação entre Idade e Presença/Faltas:</font>

A análise das porcentagens de presença e faltas por faixa etária revela padrões importantes no comportamento dos alunos em relação à frequência nas aulas:

1. **Faixa Etária 0-10 anos**:
- **Presença**: **85,2%**
- **Faltas**: **14,8%**
- **Interpretação**: Os alunos mais novos (0-10 anos) têm a maior taxa de presença. Isso pode indicar que, nessa faixa etária, há um maior controle dos responsáveis e uma maior adesão à rotina escolar.

2. **Faixa Etária 11-15 anos**:
- **Presença**: **81,8%**
- **Faltas**: **18,2%**
- **Interpretação**: A presença ainda é alta, embora um pouco menor do que a faixa anterior. Nessa idade, os alunos podem estar começando a ter mais autonomia, o que pode explicar o leve aumento nas faltas. O engajamento permanece elevado, mas já há sinais de diminuição.

3. **Faixa Etária 16-20 anos**:
- **Presença**: **74,9%**
- **Faltas**: **25,1%**
- **Interpretação**: A partir dessa faixa etária, a taxa de presença começa a cair de forma mais significativa. Esse grupo, geralmente no final do ensino médio, pode ter mais responsabilidades externas, como estágios ou preparação para vestibulares, o que pode impactar a frequência escolar. O aumento nas faltas pode também ser atribuído ao início de um comportamento mais independente.

4. **Faixa Etária 21-25 anos**:
- **Presença**: **60,4%**
- **Faltas**: **39,6%**
- **Interpretação**: A queda mais drástica na presença é observada nessa faixa etária. Alunos entre 21 e 25 anos costumam estar na transição para o mercado de trabalho ou cursando estudos superiores. Muitas vezes, conciliam estudos e trabalho, o que pode explicar o aumento significativo nas faltas. 

5. **Faixa Etária 26-30 anos**:
- **Presença**: **73,3%**
- **Faltas**: **26,7%**
- **Interpretação**: A taxa de presença volta a subir para 73,3%, o que pode indicar que os alunos dessa faixa etária que permanecem no sistema de ensino têm mais motivação e comprometimento, talvez buscando aperfeiçoamento profissional ou concluindo etapas educacionais que foram adiadas.

6. **Faixa Etária 30+ anos**:
- **Presença**: **66,0%**
- **Faltas**: **34,0%**
- **Interpretação**: A taxa de presença volta a cair nessa faixa etária, mas ainda se mantém relativamente alta. Alunos com mais de 30 anos provavelmente estão buscando educação continuada ou completando estudos interrompidos anteriormente. A taxa de faltas pode ser influenciada por obrigações pessoais e profissionais mais estabelecidas, como trabalho e família.

**Considerações Gerais:**

- **Tendência Geral**: A taxa de presença tende a diminuir à medida que a idade dos alunos aumenta, com a maior queda ocorrendo entre os alunos de **21 a 25 anos**. Isso pode refletir mudanças nas prioridades à medida que os alunos entram na vida adulta, com responsabilidades profissionais e pessoais competindo com a educação formal.

- **Exceção dos 26-30 anos**: A presença na faixa de **26-30 anos** se recupera em comparação com a faixa de 21-25 anos. Essa recuperação pode indicar que alunos nessa faixa estão mais focados em completar sua formação ou em busca de desenvolvimento profissional.

- **Implicações para Intervenção**: Programas que incentivem a retenção e o engajamento de alunos entre **16 e 25 anos** podem ser úteis para melhorar as taxas de presença. O apoio extra pode ser necessário para essa faixa, especialmente para alunos que estão entrando no mercado de trabalho ou lidando com múltiplas responsabilidades.

Esta análise demonstra uma clara correlação entre o aumento da idade e a diminuição da presença nas aulas, com algumas nuances importantes em faixas específicas que podem ser alvos de intervenção para melhorar o engajamento.

<font color='orange'>**Distribuição por Sexo:**</font>

Existem diferenças na frequência de meninos e meninas nas aulas?

In [14]:
# Contando o número de alunos por sexo
sexo_contagem = df_aluno['sexo_aluno'].value_counts()

# Dicionário para os rótulos de sexo
sexo_labels = {0: 'Masculino', 1: 'Feminino'}

# Gráfico de pizza para mostrar a distribuição de sexo
fig_sexo_pizza = px.pie(values=sexo_contagem.values, 
                        names=sexo_contagem.index.map(sexo_labels),
                        title='Distribuição de Alunos por Sexo',
                        color_discrete_sequence=px.colors.sequential.Blues_r)

# Ajustando o layout com fundo escuro
fig_sexo_pizza.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',  
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white'                
)

# Exibindo o gráfico
fig_sexo_pizza.show()


In [15]:
df_aluno['Sexo'].value_counts()

Sexo
F    1200
M    1037
Name: count, dtype: int64

In [16]:
# Juntando as tabelas de frequência e aluno para trazer o sexo dos alunos
df_sexo_freq = pd.merge(df_diarioFreq, df_aluno[['id_aluno', 'sexo_aluno']], on='id_aluno', how='inner')

# Removendo quaisquer valores nulos (se houver)
df_sexo_freq.dropna(subset=['presente', 'sexo_aluno'], inplace=True)

# Calculando a porcentagem de presença por sexo
presenca_por_sexo = df_sexo_freq.groupby('sexo_aluno')['presente'].mean() * 100

# Calculando a porcentagem de faltas por sexo
faltas_por_sexo = 100 - presenca_por_sexo

# Criando um DataFrame para presença e faltas
df_presenca_faltas = pd.DataFrame({
    'Sexo': ['Masculino', 'Feminino'],
    'Presença (%)': [presenca_por_sexo[0], presenca_por_sexo[1]],
    'Faltas (%)': [faltas_por_sexo[0], faltas_por_sexo[1]]
})

print("Porcentagem de Presença por Sexo:")
print(presenca_por_sexo)
print("\nPorcentagem de Faltas por Sexo:")
print(faltas_por_sexo)

# Transformando o DataFrame para gráfico empilhado
df_presenca_faltas_melt = df_presenca_faltas.melt(id_vars='Sexo', 
                                                  value_vars=['Presença (%)', 'Faltas (%)'], 
                                                  var_name='Status', 
                                                  value_name='Porcentagem')

# Gráfico de barras empilhadas com plotly
fig = px.bar(df_presenca_faltas_melt, 
             x='Sexo', 
             y='Porcentagem', 
             color='Status', 
             title='Porcentagem de Presença e Faltas por Sexo',
             color_discrete_sequence=['#1f77b4', '#FF7F7F'])  # Azul para presença, vermelho claro para faltas

# Ajustando o layout com fundo escuro
fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',  
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white',                
    barmode='stack'                    
)

# Exibindo o gráfico
fig.show()


Porcentagem de Presença por Sexo:
sexo_aluno
0    78.920051
1    78.475944
Name: presente, dtype: float64

Porcentagem de Faltas por Sexo:
sexo_aluno
0    21.079949
1    21.524056
Name: presente, dtype: float64


<font color='orange'>**Análise de Presença e Faltas por Sexo:**</font>

**Presença:**

- **Masculino (0)**: **78,92%**
- **Feminino (1)**: **78,48%**

A taxa de presença entre alunos do sexo masculino e feminino é bastante próxima, com uma diferença mínima de aproximadamente **0,45 pontos percentuais**. Os alunos masculinos têm uma ligeiramente maior taxa de presença, porém, a diferença é pequena o suficiente para ser considerada insignificante em termos práticos.

**Faltas:**

- **Masculino (0)**: **21,08%**
- **Feminino (1)**: **21,52%**

Da mesma forma, as porcentagens de faltas são praticamente equivalentes. Os alunos do sexo feminino apresentam uma taxa de faltas ligeiramente maior (**21,52%** comparado a **21,08%** para os alunos masculinos), mas, assim como na presença, essa diferença é muito pequena para indicar um comportamento significativamente diferente entre os sexos.

**Conclusão:**

As taxas de presença e faltas são muito similares entre os sexos, indicando que **não há uma diferença relevante** no comportamento de presença e faltas entre alunos masculinos e femininos. Isso sugere que, no contexto dessa análise, o sexo dos alunos não é um fator determinante para variações na frequência escolar. Estratégias de engajamento ou intervenção não precisam ser diferenciadas com base no sexo, já que ambos apresentam padrões muito semelhantes.

<font color='orange'>**Distribuição por Raça/Cor:**</font>

Existe uma predominância de uma raça/cor específica?


In [17]:
# Selecionando as colunas relacionadas por raça/cor
raca_colunas = [col for col in df_aluno.columns if col.startswith('raca_aluno')]

# Calculando a contagem de alunos para cada raça/cor
distribuicao_raca = df_aluno[raca_colunas].sum()

# Transformando o índice para um formato mais amigável e removendo 'None'
distribuicao_raca.index = distribuicao_raca.index.str.replace('raca_aluno_', '')

# Filtrando a categoria 'None'
distribuicao_raca = distribuicao_raca[distribuicao_raca.index != 'None']

# Mapeando as categorias para os novos rótulos
raca_mapeamento = {
    'B': 'Branco',
    'A': 'Amarelo',
    'I': 'Indígena',
    'N': 'Preto',
    'P': 'Preto',
    'R': 'Pardo'
}

# Aplicando o mapeamento de categorias
distribuicao_raca.index = distribuicao_raca.index.map(raca_mapeamento)

# Agrupando as categorias "N" e "P" como "Preto"
distribuicao_raca_agrupada = distribuicao_raca.groupby(distribuicao_raca.index).sum()

# Gráfico de barras para mostrar a distribuição por raça/cor
fig_raca = px.bar(distribuicao_raca_agrupada, 
                  x=distribuicao_raca_agrupada.index, 
                  y=distribuicao_raca_agrupada.values,
                  title='Distribuição de Alunos por Raça/Cor',
                  labels={'y': 'Número de Alunos', 'x': 'Raça/Cor'},
                  color_discrete_sequence=['#1f77b4'])

# Ajustando o layout com fundo escuro
fig_raca.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',  
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white'                
)

# Exibindo o gráfico
fig_raca.show()


<font color='orange'>**Análise da Distribuição por Raça/Cor:**</font>

A distribuição de alunos por raça/cor mostra algumas categorias com uma predominância clara e outras com presença muito pequena:

1. **Branco (1027 alunos)**: É a categoria mais representada, sugerindo que a maioria dos alunos se identifica com essa classificação.
   
2. **Pardo (865 alunos)**: A segunda categoria mais representada, com uma quantidade significativa de alunos.

3. **Preto (327 alunos)**: Essa categoria que é a soma da "P" e "N" tem uma representação menor que as duas maiores categorias.

5. **Outras Categorias (Amarelo, Indígina)**:
   - **Amarelo (17 alunos)** representa uma quantidade muito pequena de alunos.
   - **Indígina (1 aluno)** tem a menor representação possível, indicando apenas um aluno nessa categoria.

**Considerações:**

- **Concentração**: As categorias "Branco" e "Pardo" dominam a distribuição, representando a maioria dos alunos.
- **Diversidade**: A presença de categorias menores como "Amarelo", "Indígina" e "Preto" indica uma diversidade na classificação racial, mas com uma concentração evidente em poucas categorias.


In [30]:

df_aluno2 = df_aluno.drop_duplicates(subset=['id_aluno'], keep='first')
df_diarioFreq2 = df_diarioFreq.drop_duplicates(subset=['id_aluno'], keep='first')

# Mapeando as categorias de raça/cor
raca_mapeamento = {
    'B': 'Branco',
    'A': 'Amarelo',
    'I': 'Indígena',
    'N': 'Preto',
    'P': 'Preto',
    'R': 'Pardo'
}

# Filtrando as colunas de raça/cor, excluindo 'None'
raca_colunas = [col for col in df_aluno2.columns if col.startswith('raca_aluno') and col != 'raca_aluno_None']

# Juntando as tabelas de frequência e aluno para trazer a raça/cor dos alunos
df_raca_freq = pd.merge(df_diarioFreq2, df_aluno2[['id_aluno'] + raca_colunas], on='id_aluno', how='inner')

# Calculando a porcentagem de presença por raça/cor
presenca_por_raca = {}
for raca in raca_colunas:
    media_presenca = df_raca_freq.groupby(raca)['presente'].mean()[1] * 100  # A média de presença para alunos com aquela raça
    
    # Usando o mapeamento diretamente na string do nome da raça
    raca_label = raca.replace('raca_aluno_', '')
    raca_label = raca_mapeamento.get(raca_label, raca_label)  # Aplicando o mapeamento
    presenca_por_raca[raca_label] = media_presenca

# Criando o DataFrame para presença e faltas
df_presenca_raca = pd.DataFrame.from_dict(presenca_por_raca, orient='index', columns=['Presença (%)'])
df_presenca_raca['Faltas (%)'] = 100 - df_presenca_raca['Presença (%)']

# Transformando o DataFrame para o formato adequado para o gráfico
df_presenca_raca_melt = df_presenca_raca.reset_index().melt(id_vars='index', 
                                                            value_vars=['Presença (%)', 'Faltas (%)'], 
                                                            var_name='Status', 
                                                            value_name='Porcentagem')

# Gráfico de barras empilhadas para presença e faltas por raça/cor
fig_raca_freq = px.bar(df_presenca_raca_melt, 
                       x='index', 
                       y='Porcentagem', 
                       color='Status', 
                       title='Porcentagem de Presença e Faltas por Raça/Cor',
                       labels={'index': 'Raça/Cor'},
                       color_discrete_sequence=['#1f77b4', '#FF7F7F'])  # Azul para presença, vermelho claro para faltas

# Ajustando o layout com fundo escuro
fig_raca_freq.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',  
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white',               
    barmode='stack'                   
)

# Exibindo o gráfico
fig_raca_freq.show()


<font color='orange'>**Análise Breve da Correlação entre Raça/Cor e Presença/Faltas:**</font>

A análise da **presença e faltas por raça/cor** revela algumas diferenças interessantes entre os grupos:

1. **Amarelo (Presença: 93,75%, Faltas: 6,25%)**:
   - Este grupo tem uma das maiores taxas de presença, indicando um engajamento forte nas aulas.
   - A baixa taxa de faltas sugere que os alunos dessa categoria estão bastante comprometidos com suas atividades.

2. **Branco (Presença: 83,91%, Faltas: 16,09%)**:
   - A categoria mais populosa (com 1027 alunos) tem uma taxa de presença moderadamente alta, mas as faltas, com **16%**, indicam que cerca de um quinto dos alunos dessa categoria estão ausentes regularmente.

3. **Indígina (Presença: 100%, Faltas: 0%)**:
   - Embora haja apenas um aluno nessa categoria, esse aluno tem **100% de presença** e nenhuma falta, o que é um comportamento notável, mas deve ser interpretado com cautela devido ao tamanho da amostra (apenas um aluno).

4. **Preto (Presença: 85,06%, Faltas: 14,94%)**:
   - A taxa de presença e faltas nesse grupo é quase idêntica à da categoria "B", o que sugere um comportamento semelhante de engajamento e absenteísmo.

7. **Pardo (Presença: 80,76%, Faltas: 19,24%)**:
   - A presença nesse grupo é similar à das categorias "B" e "P", com uma taxa de faltas de **15%**. O comportamento de presença está dentro da média observada para os outros grupos.

**Considerações Finais:**

- **Grupo Indígina**: Apesar de ser uma amostra muito pequena, esse grupo se destaca por ter **100% de presença**. Isso pode ser um outlier devido ao tamanho reduzido da amostra.
- **Grupo Amarelo**: Com a maior taxa de presença (**93,75%**), os alunos dessa categoria parecem ser os mais assíduos.
- **Outros Grupos (Branco, Preto, Pardo)**: A maioria dos grupos tem taxas de presença na faixa de **85% a 80%** e faltas entre **15% e 20%**, o que indica um comportamento relativamente consistente entre esses alunos.

<font color='orange'>**Tratando a TbProfessor:**</font>

1. **Leitura dos Dados**: O arquivo CSV é carregado com `pd.read_csv()`, ignorando erros de linhas.
   
2. **Seleção e Renomeação de Colunas**: As colunas de interesse são selecionadas e renomeadas para facilitar a manipulação.
   
3. **Conversão de Datas**: A coluna `data_nascimento_professor` é convertida para o formato `datetime`, e a idade do professor é calculada subtraindo o ano de nascimento do ano atual.

4. **Lógica Condicional para Sexo**: A coluna `Sexo` é convertida para uma variável numérica, onde `0` representa masculino e `1` feminino.

5. **Preenchimento de Valores Nulos**: A coluna `cargo_professor` tem seus valores nulos preenchidos com `"Nenhum"`.

6. **Variáveis Dummy**: As colunas `raca_professor` e `cargo_professor` são transformadas em variáveis dummy para permitir análises categóricas.


In [23]:
# Lendo o CSV com pandas
df_professor = pd.read_csv(local_arquivos_analise + "TbProfessor/Originais anonimizados/TbProfessor.csv")

# Selecionando e renomeando as colunas
df_professor = df_professor[['IdProfessor', 'DataNascimento', 'Sexo', 'CorRaca', 'Cargo']].rename(
    columns={
        'IdProfessor': 'id_professor',
        'DataNascimento': 'data_nascimento_professor',
        'CorRaca': 'raca_professor',
        'Cargo': 'cargo_professor'
    }
)

# Convertendo a coluna de data de nascimento para datetime
df_professor['data_nascimento_professor'] = pd.to_datetime(df_professor['data_nascimento_professor'], format="%Y-%m-%d %H:%M:%S", errors='coerce')

# Calculando a idade do professor
df_professor['idade_professor'] = datetime.now().year - df_professor['data_nascimento_professor'].dt.year

# Aplicando a lógica condicional para a coluna 'Sexo'
df_professor['sexo_professor'] = df_professor['Sexo'].apply(lambda x: 0 if x == "M" else 1)

# Preenchendo valores nulos na coluna 'Cargo'
df_professor['cargo_professor'] = df_professor['cargo_professor'].fillna("Nenhum")

# Criando variáveis dummy para as colunas 'raca_professor' e 'cargo_professor'
df_professor = pd.get_dummies(df_professor, columns=['raca_professor', 'cargo_professor'])

# Convertendo as colunas 'id_professor' para o tipo (string)
df_professor['id_professor'] = df_professor['id_professor'].astype(str)

# Exibindo uma amostra de 5 linhas
df_professor.sample(5)


,id_professor,data_nascimento_professor,Sexo,idade_professor,sexo_professor,raca_professor_A,raca_professor_B,raca_professor_N,raca_professor_P,raca_professor_R,cargo_professor_Nenhum,cargo_professor_Neuropsicóloga,cargo_professor_Professor,cargo_professor_Professor Inglês,cargo_professor_Professor de Inglês,cargo_professor_Professor de Matemática,cargo_professor_Professora,cargo_professor_Professora de Inglês,cargo_professor_Professora de Matemática,cargo_professor_Professora/Coordenadora,cargo_professor_Psicóloga,cargo_professor_Psicólogo,cargo_professor_professora
31,33,2003-01-13 02:00:00,M,21,0,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False
35,37,1983-03-03 03:00:00,M,41,0,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
39,41,1995-10-16 02:00:00,F,29,1,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
22,24,1991-01-16 02:00:00,F,33,1,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False
43,45,2002-11-16 02:00:00,F,22,1,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False


<font color='orange'>Realizando os merges:</font>

1. **pd.merge**: Utilizamos o `pd.merge()` para unir as tabelas, especificando as colunas em comum (`on="id_aluno"`, `on="id_diario_aula"`, etc.).
   
2. **how="left"**: O tipo de junção utilizado é o `left join`, o que significa que todos os registros da tabela à esquerda (neste caso, `df_diarioFreq` inicialmente) serão mantidos, mesmo que não haja correspondência nas tabelas à direita.

3. **Ordem dos Merges**: A junção é feita passo a passo, primeiro unindo `df_diarioFreq` com `df_aluno`, e depois adicionando as demais tabelas (como `df_diarioAula`, `df_professor`, etc.).

4. **Verificação do Resultado**: Ao final, uma amostra de 5 registros é exibida para verificar o resultado.

Este código consolidará todas as informações das tabelas em um único DataFrame (`df_raw`), deixando todos os dados unificados para análises mais completas.

In [24]:
# Realizando os merges (junção) usando pandas
df_raw = pd.merge(df_diarioFreq, df_aluno, on="id_aluno", how="left")
df_raw = pd.merge(df_raw, df_diarioAula, on="id_diario_aula", how="left")
df_raw = pd.merge(df_raw, df_professor, on="id_professor", how="left")
df_raw = pd.merge(df_raw, df_diario, on="id_diario", how="left")
df_raw = pd.merge(df_raw, df_turma, on="id_turma", how="left")

# Exibindo uma amostra dos dados após o merge
df_raw.sample(5)


,id_diario_aula,id_aluno,StPresencaFalta,presente,data_nascimento_aluno,Sexo_x,IdPai,IdMae,CorRaca,idade_aluno,sexo_aluno,tem_pai,tem_mae,raca_aluno_A,raca_aluno_B,raca_aluno_I,raca_aluno_N,raca_aluno_None,raca_aluno_P,raca_aluno_R,id_diario,data_aula,num_aula,conteudo_ministrado,id_professor,dia_semana_0,dia_semana_1,dia_semana_2,dia_semana_3,dia_semana_4,dia_semana_5,dia_semana_6,data_nascimento_professor,Sexo_y,idade_professor,sexo_professor,raca_professor_A,raca_professor_B,raca_professor_N,raca_professor_P,raca_professor_R,cargo_professor_Nenhum,cargo_professor_Neuropsicóloga,cargo_professor_Professor,cargo_professor_Professor Inglês,cargo_professor_Professor de Inglês,cargo_professor_Professor de Matemática,cargo_professor_Professora,cargo_professor_Professora de Inglês,cargo_professor_Professora de Matemática,cargo_professor_Professora/Coordenadora,cargo_professor_Psicóloga,cargo_professor_Psicólogo,cargo_professor_professora,id_turma,id_disciplina,id_serie,id_periodo,TurnoPrincipal,turno_turma
259398,21172,1535,P,1,2012-10-04 00:00:00,F,1298.0,NaN,R,12.0,1.0,1.0,0.0,False,False,False,False,False,False,True,2142,2024-03-12,3,Atividades de autoconhecimento,13,False,True,False,False,False,False,False,1976-06-08 03:00:00,F,48.0,1.0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,631,19,8,7,M,0
180356,14524,1177,P,1,2012-03-12 00:00:00,M,NaN,964,B,12.0,0.0,0.0,1.0,False,True,False,False,False,False,False,1361,2023-05-15,2,Procrastinação \r\nO que é? Como Parar de proc...,9,True,False,False,False,False,False,False,1977-01-09 03:00:00,F,47.0,1.0,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,489,21,15,6,Z,3
66166,5776,988,P,1,2005-08-10 03:00:00,F,NaN,817,B,19.0,1.0,0.0,1.0,False,True,False,False,False,False,False,463,2022-05-02,9,CORREÇÃO DA P1,8,True,False,False,False,False,False,False,1983-05-11 03:00:00,M,41.0,0.0,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,409,4,24,5,M,0
38036,3475,240,P,1,2008-10-19 00:00:00,M,NaN,229,R,16.0,0.0,0.0,1.0,False,False,False,False,False,False,True,138,2021-11-26,15,Avaliação Diagnóstica de Língua Portuguesa (Se...,12,False,False,False,False,True,False,False,1978-03-02 03:00:00,F,46.0,1.0,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,100,1,11,4,M,0
164156,13201,1627,P,1,2006-07-03 00:00:00,F,1303.0,NaN,R,18.0,1.0,1.0,0.0,False,False,False,False,False,False,True,1136,2023-04-10,4,Ortografia (sinônimos e antônimos),auticorreção,True,False,False,False,False,False,False,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,522,1,24,6,Z,3


In [25]:

# Selecionando as colunas
df = df_raw[[
    # Aula
    'data_aula',
    'dia_semana_0',
    'dia_semana_1',
    'dia_semana_2',
    'dia_semana_3',
    'dia_semana_4',
    'dia_semana_5',
    'dia_semana_6',
    'num_aula',
    'conteudo_ministrado',
    
    # Aluno
    'id_aluno',
    'sexo_aluno',
    'tem_pai',
    'tem_mae',
    'raca_aluno_A',
    'raca_aluno_B',
    'raca_aluno_I',
    'raca_aluno_N',
    'raca_aluno_None',
    'raca_aluno_P',
    'raca_aluno_R',
    'idade_aluno',
    
    # Professor
    'id_professor',
    'sexo_professor',
    'raca_professor_A',
    'raca_professor_B',
    'raca_professor_N',
    'raca_professor_P',
    'raca_professor_R',
    'cargo_professor_Nenhum',
    'cargo_professor_Neuropsicóloga',
    'cargo_professor_Professor',
    'cargo_professor_Professor Inglês',
    'cargo_professor_Professor de Inglês',
    'cargo_professor_Professor de Matemática',
    'cargo_professor_Professora',
    'cargo_professor_Professora de Inglês',
    'cargo_professor_Professora de Matemática',
    'cargo_professor_Professora/Coordenadora',
    'cargo_professor_Psicóloga',
    'cargo_professor_Psicólogo',
    'cargo_professor_professora',
    'idade_professor',
    
    # Turma
    'id_turma',
    'id_disciplina',
    'id_serie',
    'id_periodo',
    'turno_turma',
    
    # Target
    'presente'
]]

# Exibindo uma amostra do DataFrame selecionado
df.sample(5)


,data_aula,dia_semana_0,dia_semana_1,dia_semana_2,dia_semana_3,dia_semana_4,dia_semana_5,dia_semana_6,num_aula,conteudo_ministrado,id_aluno,sexo_aluno,tem_pai,tem_mae,raca_aluno_A,raca_aluno_B,raca_aluno_I,raca_aluno_N,raca_aluno_None,raca_aluno_P,raca_aluno_R,idade_aluno,id_professor,sexo_professor,raca_professor_A,raca_professor_B,raca_professor_N,raca_professor_P,raca_professor_R,cargo_professor_Nenhum,cargo_professor_Neuropsicóloga,cargo_professor_Professor,cargo_professor_Professor Inglês,cargo_professor_Professor de Inglês,cargo_professor_Professor de Matemática,cargo_professor_Professora,cargo_professor_Professora de Inglês,cargo_professor_Professora de Matemática,cargo_professor_Professora/Coordenadora,cargo_professor_Psicóloga,cargo_professor_Psicólogo,cargo_professor_professora,idade_professor,id_turma,id_disciplina,id_serie,id_periodo,turno_turma,presente
227000,2023-09-21,False,False,False,True,False,False,False,8,AVALIAÇÃO DE PORTUGUÊS (P3).,1167,1.0,0.0,1.0,False,True,False,False,False,False,False,16.0,5,1.0,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,55.0,580,1,25,6,0,1
242745,2023-11-07,False,True,False,False,False,False,False,6,Planejar e executar pesquisa amostral sobre qu...,280,0.0,0.0,1.0,False,True,False,False,False,False,False,15.0,8,0.0,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,41.0,516,4,25,6,0,1
124106,2022-09-21,False,False,True,False,False,False,False,8,AVALIAÇÃAO P3- PRIMEIRA PARTE- REDAÇÃO PLATAFO...,515,0.0,0.0,1.0,False,False,False,False,False,False,True,15.0,5,1.0,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,55.0,205,1,25,5,3,1
38918,2021-11-30,False,True,False,False,False,False,False,16,Aula de Encerramento: preenchimento dos formul...,404,1.0,1.0,0.0,False,True,False,False,False,False,False,13.0,9,1.0,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,47.0,74,4,8,4,1,1
290713,2024-05-17,False,False,False,False,True,False,False,3,Tema: Psico – Mapa da empatia,586,1.0,0.0,1.0,False,False,False,False,False,False,True,14.0,26,0.0,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,54.0,716,22,25,7,0,1


In [26]:
print(df['data_aula'].dt.year.unique())
print(df.shape)

[2021 2022 2023 2024]
(313160, 49)


<font color='orange'>**TESTE Função buscar aluno com o PANDAS**</font>

In [47]:
df_prox = df.sort_values(['id_aluno', 'data_aula']).copy()

# Criando a coluna 'seq_presenca' com a sequência ordinal de presença para cada aluno
df_prox['seq_presenca'] = df_prox.groupby('id_aluno').cumcount() + 1

df_prox.head()

,data_aula,dia_semana_0,dia_semana_1,dia_semana_2,dia_semana_3,dia_semana_4,dia_semana_5,dia_semana_6,num_aula,conteudo_ministrado,id_aluno,sexo_aluno,tem_pai,tem_mae,raca_aluno_A,raca_aluno_B,raca_aluno_I,raca_aluno_N,raca_aluno_None,raca_aluno_P,raca_aluno_R,idade_aluno,id_professor,sexo_professor,raca_professor_A,raca_professor_B,raca_professor_N,raca_professor_P,raca_professor_R,cargo_professor_Nenhum,cargo_professor_Neuropsicóloga,cargo_professor_Professor,cargo_professor_Professor Inglês,cargo_professor_Professor de Inglês,cargo_professor_Professor de Matemática,cargo_professor_Professora,cargo_professor_Professora de Inglês,cargo_professor_Professora de Matemática,cargo_professor_Professora/Coordenadora,cargo_professor_Psicóloga,cargo_professor_Psicólogo,cargo_professor_professora,idade_professor,id_turma,id_disciplina,id_serie,id_periodo,turno_turma,presente,seq_presenca
178961,2023-05-11,False,False,False,True,False,False,False,2,"Reconhecimento das emoções , nomeie as emoções...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,1.0,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,41.0,466,19,8,6,1,1,1
179553,2023-05-12,False,False,False,False,True,False,False,2,Subtração \r\nRelembrando igualdade,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1.0,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,47.0,466,4,8,6,1,1,2
6121,2021-07-27,False,True,False,False,False,False,False,1,Live de boas vindas,3,1.0,0.0,1.0,False,False,False,False,False,False,True,19.0,6,0.0,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,34.0,106,4,29,4,1,1,1
4095,2021-07-29,False,False,False,True,False,False,False,1,Live de retorno 2 semestre com Acolhimento da ...,3,1.0,0.0,1.0,False,False,False,False,False,False,True,19.0,3,1.0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,39.0,106,1,29,4,1,1,2
4622,2021-08-03,False,True,False,False,False,False,False,1,Retomada de Planejamento.,3,1.0,0.0,1.0,False,False,False,False,False,False,True,19.0,7,1.0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,44.0,106,17,29,4,1,1,3


In [68]:
def prepare_datos_predict(df: pd.DataFrame, id_aluno):

    df = df.sort_values(['id_aluno', 'data_aula']).copy()

    # Criando a coluna 'seq_presenca' com a sequência ordinal de presença para cada aluno
    df['seq_presenca'] = df.groupby('id_aluno').cumcount() + 1

    # Filtrando o dataframe para o id_aluno específico
    df_aluno = df[df["id_aluno"] == id_aluno]

    # Calculando as janelas de presença
    inicio_janela = df_aluno["seq_presenca"].max() - 10
    fim_janela = df_aluno["seq_presenca"].max()

    # Carregando o modelo de NLP
    nlp = spacy.load("pt_core_news_sm")

    # Função para criar embeddings
    def criar_embedding(texto):
        doc = nlp(texto)
        return doc.vector.tolist()

    # Filtrando o dataframe nas janelas de presença
    df_janela = df_aluno[(df_aluno["seq_presenca"] >= inicio_janela) & (df_aluno["seq_presenca"] <= fim_janela)]

    # Agrupando os dados e criando as agregações necessárias
    df_agg = df_janela.groupby("id_aluno").agg({
        "data_aula": ["first", "last"],
        "dia_semana_0": "sum",
        "dia_semana_1": "sum",
        "dia_semana_2": "sum",
        "dia_semana_3": "sum",
        "dia_semana_4": "sum",
        "dia_semana_5": "sum",
        "dia_semana_6": "sum",
        "conteudo_ministrado": lambda x: " ".join(x),  # Concatenando o conteúdo
        "sexo_aluno": "first",
        "tem_pai": "first",
        "tem_mae": "first",
        "raca_aluno_A": "first",
        "raca_aluno_B": "first",
        "raca_aluno_I": "first",
        "raca_aluno_N": "first",
        "raca_aluno_None": "first",
        "raca_aluno_P": "first",
        "raca_aluno_R": "first",
        "idade_aluno": "mean",
        "sexo_professor": "sum",
        "raca_professor_A": "sum",
        "raca_professor_B": "sum",
        "raca_professor_N": "sum",
        "raca_professor_P": "sum",
        "raca_professor_R": "sum",
        "cargo_professor_Nenhum": "sum",
        "cargo_professor_Neuropsicóloga": "sum",
        "cargo_professor_Professor": "sum",
        "cargo_professor_Professor Inglês": "sum",
        "cargo_professor_Professor de Inglês": "sum",
        "cargo_professor_Professor de Matemática": "sum",
        "cargo_professor_Professora": "sum",
        "cargo_professor_Professora de Inglês": "sum",
        "cargo_professor_Professora de Matemática": "sum",
        "cargo_professor_Professora/Coordenadora": "sum",
        "cargo_professor_Psicóloga": "sum",
        "cargo_professor_Psicólogo": "sum",
        "idade_professor": "mean",
        "id_professor": pd.Series.nunique,  # Número de professores diferentes
        "id_disciplina": pd.Series.nunique,  # Número de disciplinas diferentes
        "id_serie": pd.Series.nunique,  # Número de séries diferentes
        "id_turma": pd.Series.nunique,  # Número de turmas diferentes
        "presente": ["sum", "count"],  # Soma e contagem de presenças
        "seq_presenca": lambda x: x[df_janela["presente"] == 1].max()  # Última presença
    })

    # Ajustando os nomes das colunas agregadas
    df_agg.columns = ["_".join(col).strip() if isinstance(col, tuple) else col for col in df_agg.columns]

    # Renomeando as colunas geradas pelo `groupby` para seguir o padrão desejado
    df_agg.rename(columns={
        'conteudo_ministrado_<lambda>': 'conteudo_ministrado_concat',
        'seq_presenca_<lambda>': 'ultima_presenca',
        'dia_semana_0_sum': 'sum_dia_semana_1',
        'dia_semana_1_sum': 'sum_dia_semana_2',
        'dia_semana_2_sum': 'sum_dia_semana_3',
        'dia_semana_3_sum': 'sum_dia_semana_4',
        'dia_semana_4_sum': 'sum_dia_semana_5',
        'dia_semana_5_sum': 'sum_dia_semana_6',
        'dia_semana_6_sum': 'sum_dia_semana_7',
        'sexo_aluno_first': 'sexo_aluno',
        'tem_pai_first': 'tem_pai',
        'tem_mae_first': 'tem_mae',
        'raca_aluno_A_first': 'raca_aluno_A',
        'raca_aluno_B_first': 'raca_aluno_B',
        'raca_aluno_I_first': 'raca_aluno_I',
        'raca_aluno_N_first': 'raca_aluno_N',
        'raca_aluno_None_first': 'raca_aluno_None',
        'raca_aluno_P_first': 'raca_aluno_P',
        'raca_aluno_R_first': 'raca_aluno_R',
        'idade_aluno_mean': 'idade_aluno_media',
        'cargo_professor_Professor Inglês_sum': 'cargo_professor_Professor_Inglês_sum',
        'cargo_professor_Professor de Inglês_sum': 'cargo_professor_Professor_de_Inglês_sum',
        'cargo_professor_Professor de Matemática_sum': 'cargo_professor_Professor_de_Matemática_sum',
        'cargo_professor_Professora de Inglês_sum': 'cargo_professor_Professora_de_Inglês_sum',
        'cargo_professor_Professora de Matemática_sum': 'cargo_professor_Professora_de_Matemática_sum',
        'cargo_professor_Professora/Coordenadora_sum': 'cargo_professor_Professora_Coordenadora_sum',
        'idade_professor_mean': 'idade_professor_media',
        'id_professor_nunique': 'num_professores',
        'id_disciplina_nunique': 'num_disciplinas',
        'id_serie_nunique': 'num_series',
        'id_turma_nunique': 'num_turmas',
        'presente_sum': 'num_presencas',
        'presente_count': 'num_aulas'
    }, inplace=True)

    # Adicionando a coluna 'last_seq'
    df_agg["last_seq"] = fim_janela

    # Criando os embeddings para o conteúdo ministrado
    df_agg["conteudo_embedding"] = df_agg["conteudo_ministrado_concat"].apply(criar_embedding)

    # Remove a coluna 'conteudo_embedding' antiga e expande os embeddings
    abt_filtered = df_agg.drop(columns=['conteudo_embedding', 'conteudo_ministrado_concat'])

    # Expande a lista de embeddings (conteúdo_embedding) em múltiplas colunas
    conteudo_embedding_expanded = pd.DataFrame(df_agg['conteudo_embedding'].tolist(), index=df_agg.index)

    # Converte as colunas numéricas de embeddings para strings representando números
    conteudo_embedding_expanded.columns = [str(i) for i in range(conteudo_embedding_expanded.shape[1])]

    # Concatena o DataFrame original com as novas colunas de embedding
    abt_final = pd.concat([abt_filtered, conteudo_embedding_expanded], axis=1)

    # Preenchendo os valores NaN da coluna 'ultima_presenca' com a média de 'num_presenca'
    media_presenca = abt_final['num_presencas'].mean()
    abt_final['ultima_presenca'].fillna(media_presenca, inplace=True)

    # Retornando o dataframe final
    return abt_final.reset_index()

# Testando a função
id_aluno = np.int64(102)
df_janela2 = prepare_datos_predict(df, id_aluno)
df_janela2


,id_aluno,data_aula_first,data_aula_last,sum_dia_semana_1,sum_dia_semana_2,sum_dia_semana_3,sum_dia_semana_4,sum_dia_semana_5,sum_dia_semana_6,sum_dia_semana_7,sexo_aluno,tem_pai,tem_mae,raca_aluno_A,raca_aluno_B,raca_aluno_I,raca_aluno_N,raca_aluno_None,raca_aluno_P,raca_aluno_R,idade_aluno_media,sexo_professor_sum,raca_professor_A_sum,raca_professor_B_sum,raca_professor_N_sum,raca_professor_P_sum,raca_professor_R_sum,cargo_professor_Nenhum_sum,cargo_professor_Neuropsicóloga_sum,cargo_professor_Professor_sum,cargo_professor_Professor_Inglês_sum,cargo_professor_Professor_de_Inglês_sum,cargo_professor_Professor_de_Matemática_sum,cargo_professor_Professora_sum,cargo_professor_Professora_de_Inglês_sum,cargo_professor_Professora_de_Matemática_sum,cargo_professor_Professora_Coordenadora_sum,cargo_professor_Psicóloga_sum,cargo_professor_Psicólogo_sum,idade_professor_media,num_professores,num_disciplinas,num_series,num_turmas,num_presencas,num_aulas,ultima_presenca,last_seq,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
0,102,2022-06-07,2022-06-30,0,4,4,3,0,0,0,1.0,0.0,1.0,False,False,False,False,False,False,True,19.0,6.0,0,8,0,0,3,0,0,5,0,0,0,4,0,0,0,2,0,48.181818,4,4,2,2,0,11,0.0,123,0.114647,-1.40983,-0.017523,1.246244,0.054283,0.797433,0.768126,0.926144,0.525275,0.203207,0.407622,-2.665226,-1.500093,0.767286,-1.899162,1.956527,1.249517,0.709662,0.449481,0.213025,0.190514,0.574134,-0.09262,0.465144,-1.194155,-0.565527,1.010417,0.669021,1.697539,0.898498,0.008078,1.067726,0.511524,-0.445266,0.194692,-0.325014,-0.870756,0.111917,-0.173054,-0.332524,-1.408381,1.981254,-0.453415,1.701112,0.187319,0.567705,-1.27237,1.034499,-1.054332,-1.675773,-1.522119,0.143676,1.513889,0.270373,0.854043,0.160905,0.721267,-0.737817,-1.946351,-1.13472,-0.408238,-1.027617,-1.341581,-1.122147,0.335029,1.180084,-0.416602,-1.029512,-0.383877,0.796493,-1.373293,1.048338,0.86538,-1.984326,0.543008,0.158557,-0.969629,0.523497,1.40189,-2.082065,0.470857,1.783062,-1.006254,1.290781,0.814985,0.724475,0.837011,-0.379999,-1.64262,1.107531,0.207118,-1.62815,2.241691,1.90668,-1.611591,0.10455


In [24]:
# df.to_csv('./documents/base dados/dados_gerais_passos_magicos.csv', index=False)

<font color='orange'>**Correlação entre Pais (IdPai, IdMae) e Presença**</font>

Alunos que têm ambos os pais cadastrados faltam menos? Isso pode revelar fatores sociais que impactam a presença.

In [112]:
# Criando uma nova coluna para categorizar os alunos
df['categoria_pais'] = df.apply(lambda row: 'Ambos Pais' if row['tem_pai'] == 1 and row['tem_mae'] == 1 
                                 else 'Só Pai ou Só Mãe' if row['tem_pai'] != row['tem_mae'] 
                                 else 'Sem Pais', axis=1)

# Calculando a média de presença para cada categoria
presenca_categoria = df.groupby('categoria_pais')['presente'].mean() * 100
faltas_categoria = 100 - presenca_categoria

# Criando DataFrame para visualização dos resultados
df_categoria_pais = pd.DataFrame({
    'Categoria': ['Ambos Pais', 'Só Pai ou Só Mãe', 'Sem Pais'],
    'Presença (%)': presenca_categoria.values,
    'Faltas (%)': faltas_categoria.values
})

# Transformando o DataFrame para gráfico empilhado
df_categoria_pais_melt = df_categoria_pais.melt(id_vars='Categoria', 
                                                value_vars=['Presença (%)', 'Faltas (%)'], 
                                                var_name='Status', 
                                                value_name='Porcentagem')

# Gráfico de barras empilhadas para presença e faltas por categoria
fig_categoria_pais = px.bar(df_categoria_pais_melt, 
                            x='Categoria', 
                            y='Porcentagem', 
                            color='Status', 
                            title='Presença e Faltas por Categoria de Pais',
                            color_discrete_sequence=['#1f77b4', '#FF7F7F'])  # Azul para presença, vermelho claro para faltas

# Ajustando o layout com fundo escuro
fig_categoria_pais.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',  
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white',               
    barmode='stack'                   
)

# Exibindo o gráfico
fig_categoria_pais.show()

# Exibindo os valores para análise
print(df_categoria_pais)


          Categoria  Presença (%)  Faltas (%)
0        Ambos Pais     73.847803   26.152197
1  Só Pai ou Só Mãe     77.021717   22.978283
2          Sem Pais     78.741196   21.258804


<font color='orange'>**Análise da Correlação entre Presença de Pais e Presença/Faltas:**</font>

1. **Alunos com Ambos os Pais**:
   - **Presença**: **73,85%**
   - **Faltas**: **26,15%**
   - **Interpretação**: Curiosamente, alunos que têm ambos os pais registrados apresentam a **menor taxa de presença** e a maior taxa de faltas (**26,15%**). Isso pode indicar que a presença de ambos os pais não necessariamente garante maior assiduidade nas aulas.

2. **Alunos com Apenas Pai ou Mãe**:
   - **Presença**: **77,02%**
   - **Faltas**: **22,98%**
   - **Interpretação**: Alunos que têm apenas pai ou mãe registrados apresentam uma taxa de presença maior (**77,02%**), e a taxa de faltas diminui para cerca de **23%**. Essa categoria parece estar em uma situação intermediária entre alunos com ambos os pais e sem pais.

3. **Alunos Sem Pais**:
   - **Presença**: **78,74%**
   - **Faltas**: **21,26%**
   - **Interpretação**: De maneira contraintuitiva, alunos sem pai nem mãe registrados têm a **maior taxa de presença** (**78,74%**) e a menor taxa de faltas. Isso sugere que, para esses alunos, a falta de pais registrados não resulta em uma menor presença escolar, e pode haver outros fatores sociais ou institucionais que motivam esses alunos a comparecerem mais frequentemente.

**Conclusão:**

- **Alunos com ambos os pais** têm a maior taxa de faltas, o que é um resultado inesperado. Intervenções para engajar mais esses alunos podem ser úteis.
- **Alunos sem pais** se destacam por ter a maior taxa de presença, sugerindo que esses alunos podem receber suporte adicional ou possuem uma motivação interna maior para comparecer às aulas.
- **Alunos com apenas um dos pais** estão em uma situação intermediária, com uma boa taxa de presença, indicando que a presença de pelo menos um dos pais pode ajudar no engajamento escolar.

Esses resultados indicam que a presença de pais, por si só, não é o único fator determinante para a assiduidade, e outras variáveis podem estar influenciando o comportamento dos alunos.

In [113]:
# Calculando a média de presença para alunos com e sem pai
presenca_pai = df.groupby('tem_pai')['presente'].mean() * 100
faltas_pai = 100 - presenca_pai

# Calculando a média de presença para alunos com e sem mãe
presenca_mae = df.groupby('tem_mae')['presente'].mean() * 100
faltas_mae = 100 - presenca_mae

# Criando DataFrame para visualização dos resultados
df_pai_mae = pd.DataFrame({
    'Grupo': ['Tem Pai', 'Não Tem Pai', 'Tem Mãe', 'Não Tem Mãe'],
    'Presença (%)': [presenca_pai[1], presenca_pai[0], presenca_mae[1], presenca_mae[0]],
    'Faltas (%)': [faltas_pai[1], faltas_pai[0], faltas_mae[1], faltas_mae[0]]
})

# Transformando o DataFrame para gráfico empilhado
df_pai_mae_melt = df_pai_mae.melt(id_vars='Grupo', 
                                  value_vars=['Presença (%)', 'Faltas (%)'], 
                                  var_name='Status', 
                                  value_name='Porcentagem')

# Gráfico de barras empilhadas para presença e faltas por grupo (pai/mãe)
fig_pai_mae = px.bar(df_pai_mae_melt, 
                     x='Grupo', 
                     y='Porcentagem', 
                     color='Status', 
                     title='Presença e Faltas por Presença de Pai/Mãe',
                     color_discrete_sequence=['#1f77b4', '#FF7F7F'])  # Azul para presença, vermelho claro para faltas

# Ajustando o layout com fundo escuro
fig_pai_mae.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',  
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white',               
    barmode='stack'                   
)

# Exibindo o gráfico
fig_pai_mae.show()

# Exibindo os valores para análise
print(df_pai_mae)


         Grupo  Presença (%)  Faltas (%)
0      Tem Pai     85.761384   14.238616
1  Não Tem Pai     76.600463   23.399537
2      Tem Mãe     76.572006   23.427994
3  Não Tem Mãe     84.840616   15.159384


<font color='orange'>**Análise da Correlação entre Presença de Pais/Mães e Presença/Faltas:**</font>

1. **Alunos com Pai**:
   - **Presença**: **85,76%**
   - **Faltas**: **14,24%**
   - **Interpretação**: Alunos que têm pai registrado apresentam uma alta taxa de presença (**85,76%**), sugerindo um maior comprometimento com as aulas em comparação com os alunos sem pai registrado.

2. **Alunos sem Pai**:
   - **Presença**: **76,60%**
   - **Faltas**: **23,40%**
   - **Interpretação**: A ausência de pai registrado está associada a uma redução significativa na presença, com quase **23,4% de faltas**, indicando um possível fator de vulnerabilidade.

3. **Alunos com Mãe**:
   - **Presença**: **76,57%**
   - **Faltas**: **23,43%**
   - **Interpretação**: A presença de mãe registrada não parece influenciar tanto quanto a de pai. Alunos com mãe têm uma taxa de presença ligeiramente inferior, similar à dos alunos sem pai.

4. **Alunos sem Mãe**:
   - **Presença**: **84,84%**
   - **Faltas**: **15,16%**
   - **Interpretação**: Curiosamente, alunos sem mãe registrada apresentam uma presença mais alta (**84,84%**) do que aqueles com mãe registrada. Esse comportamento pode ser influenciado por outros fatores que exigem uma investigação mais profunda.

**Conclusão:**

- **Presença de Pai**: A presença de um pai registrado está associada a uma maior taxa de presença. A falta de pai registrado parece impactar negativamente a presença dos alunos, aumentando significativamente as faltas.
- **Presença de Mãe**: A diferença entre ter ou não mãe registrada é menos clara, com alunos sem mãe apresentando uma taxa de presença até superior àqueles com mãe registrada.

Esses dados sugerem que a **ausência de pai** tem um impacto mais significativo na frequência escolar.

<font color='orange'>**Correlação entre a Disciplina e Presença**</font>

Quais disciplinas precisam de ajustes para melhorar a participação dos alunos e quais estão funcionando bem?

In [114]:
df_disciplina = pd.read_csv(local_arquivos_analise + "TbDisciplina/Originais anonimizados/TbDisciplina.csv")

df_disciplina = df_disciplina[['IdDisciplina', 'NomeDisciplina']]

df_disciplina.columns = ['id_disciplina', 'NomeDisciplina']

df_disciplina

,id_disciplina,NomeDisciplina
0,1,PORTUGUÊS
1,4,MATEMÁTICA
2,5,INGLÊS
3,16,HERÓIS DA EDUCAÇÃO
4,17,SACODE A POEIRA
5,18,POLIVALENTE
6,19,SABEDORIA EM AÇÃO
7,20,TRILHANDO MEU CAMINHO
8,21,JORNADA DAS EMOÇÕES
9,22,QUEBRANDO BARREIRAS


In [115]:
# Realizando o merge para trazer o nome da disciplina
df_merged = pd.merge(df, df_disciplina, on='id_disciplina', how='left')

# Calculando a média de presença por disciplina
presenca_disciplina = df_merged.groupby('NomeDisciplina')['presente'].mean() * 100
faltas_disciplina = 100 - presenca_disciplina

# Criando DataFrame para visualização dos resultados
df_disciplina_presenca = pd.DataFrame({
    'Disciplina': presenca_disciplina.index,
    'Presença (%)': presenca_disciplina.values,
    'Faltas (%)': faltas_disciplina.values
})

# Transformando o DataFrame para gráfico empilhado
df_disciplina_presenca_melt = df_disciplina_presenca.melt(id_vars='Disciplina', 
                                                          value_vars=['Presença (%)', 'Faltas (%)'], 
                                                          var_name='Status', 
                                                          value_name='Porcentagem')

# Gráfico de barras empilhadas para presença e faltas por disciplina
fig_disciplina = px.bar(df_disciplina_presenca_melt, 
                        x='Disciplina', 
                        y='Porcentagem', 
                        color='Status', 
                        title='Presença e Faltas por Disciplina',
                        color_discrete_sequence=['#1f77b4', '#FF7F7F'])  # Azul para presença, vermelho claro para faltas

# Ajustando o layout com fundo escuro
fig_disciplina.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)', 
    paper_bgcolor='rgba(0, 0, 0, 0)', 
    font_color='white',               
    barmode='stack'                   
)

# Exibindo o gráfico
fig_disciplina.show()

# Exibindo os valores para análise
print(df_disciplina_presenca)


                                     Disciplina  Presença (%)  Faltas (%)
0   EU NO COMANDO / SACODE A POEIRA / SUPERAÇÃO     80.264218   19.735782
1                            GUARDIÕES DO SABER     86.301522   13.698478
2                            HERÓIS DA EDUCAÇÃO     78.614872   21.385128
3                                        INGLÊS     75.463448   24.536552
4                           JORNADA DAS EMOÇÕES     87.945276   12.054724
5                                    MATEMÁTICA     82.606091   17.393909
6                                   POLIVALENTE     49.157136   50.842864
7                                     PORTUGUÊS     82.888098   17.111902
8                           QUEBRANDO BARREIRAS     79.541072   20.458928
9                             SABEDORIA EM AÇÃO     77.852650   22.147350
10                              SACODE A POEIRA     71.116917   28.883083
11                                    SUPERAÇÃO    100.000000    0.000000
12                        TRILHANDO ME

#### <font color='orange'>**Análise da Presença e Faltas por Disciplina:**</font>

1. **JORNADA DAS EMOÇÕES**:
   - **Presença**: **87,95%**
   - **Faltas**: **12,05%**
   - **Interpretação**: A disciplina com a maior taxa de presença, indicando um alto nível de engajamento dos alunos. Com uma taxa de faltas bem baixa, é uma das disciplinas com o melhor desempenho em termos de frequência.

2. **GUARDIÕES DO SABER**:
   - **Presença**: **86,30%**
   - **Faltas**: **13,70%**
   - **Interpretação**: Também apresenta uma taxa de presença elevada, com uma baixa taxa de faltas, mostrando bom engajamento.

3. **SUPERAÇÃO**:
   - **Presença**: **100%**
   - **Faltas**: **0%**
   - **Interpretação**: Notavelmente, todos os alunos registrados assistiram a essa disciplina, resultando em **100% de presença**. Isso pode indicar uma disciplina de curta duração ou um tema altamente motivacional.

4. **MATEMÁTICA** e **PORTUGUÊS**:
   - **Presença**: **82,61%** e **82,89%**, respectivamente.
   - **Faltas**: **17,39%** e **17,11%**.
   - **Interpretação**: Essas disciplinas apresentam um desempenho sólido em termos de presença, com taxas de faltas moderadas, típicas para disciplinas centrais no currículo.

5. **INGLÊS**:
   - **Presença**: **75,46%**
   - **Faltas**: **24,54%**
   - **Interpretação**: Uma das disciplinas com menor taxa de presença, sugerindo que os alunos podem ter mais dificuldades ou menos interesse em comparecer às aulas de inglês.

6. **POLIVALENTE**:
   - **Presença**: **49,16%**
   - **Faltas**: **50,84%**
   - **Interpretação**: Essa disciplina tem uma das piores taxas de presença, com mais da metade dos alunos faltando. A alta taxa de faltas pode indicar falta de engajamento ou dificuldades associadas ao conteúdo.

7. **TRILHANDO MEU CAMINHO**:
   - **Presença**: **27,74%**
   - **Faltas**: **72,26%**
   - **Interpretação**: Esta é a disciplina com a **pior taxa de presença**. Mais de 70% dos alunos estão ausentes, o que indica que há uma necessidade urgente de reavaliar a forma como essa disciplina está sendo conduzida ou as condições que estão afetando o comparecimento.

8. **DISCIPLINAS MOTIVACIONAIS** (e.g., "EU NO COMANDO / SACODE A POEIRA / SUPERAÇÃO"):
   - **Presença**: **80,26%** a **100%**
   - **Interpretação**: As disciplinas com temas motivacionais têm uma presença geralmente alta, sugerindo que o conteúdo inspira os alunos a participar mais.

**Conclusão:**

- **Disciplinas de Engajamento Alto**: "Jornada das Emoções", "Guardião dos Saberes" e "Superação" estão no topo da lista, com taxas de presença acima de 85%, indicando que essas disciplinas são populares e bem recebidas pelos alunos.
- **Áreas de Preocupação**: Disciplinas como "Polivalente" e "Trilhando Meu Caminho" têm taxas de faltas extremamente altas, exigindo uma revisão do conteúdo ou estratégias para aumentar a participação dos alunos.
- **Disciplinas Centrais (Matemática, Português)**: Essas disciplinas têm boas taxas de presença, mas ainda podem se beneficiar de estratégias que reduzam as faltas e aumentem o engajamento.



### <font color='orange'>Criando Safras e ABT:</font>

In [116]:
df_prox = df.sort_values(['id_aluno', 'data_aula']).copy()

# Criando a coluna 'seq_presenca' com a sequência ordinal de presença para cada aluno
df_prox['seq_presenca'] = df_prox.groupby('id_aluno').cumcount() + 1

In [118]:
# Exibir o dataframe com a nova coluna
df_prox.sample(5)

,data_aula,dia_semana_0,dia_semana_1,dia_semana_2,dia_semana_3,dia_semana_4,dia_semana_5,dia_semana_6,num_aula,conteudo_ministrado,id_aluno,sexo_aluno,tem_pai,tem_mae,raca_aluno_A,raca_aluno_B,raca_aluno_I,raca_aluno_N,raca_aluno_None,raca_aluno_P,raca_aluno_R,idade_aluno,id_professor,sexo_professor,raca_professor_A,raca_professor_B,raca_professor_N,raca_professor_P,raca_professor_R,cargo_professor_Nenhum,cargo_professor_Neuropsicóloga,cargo_professor_Professor,cargo_professor_Professor Inglês,cargo_professor_Professor de Inglês,cargo_professor_Professor de Matemática,cargo_professor_Professora,cargo_professor_Professora de Inglês,cargo_professor_Professora de Matemática,cargo_professor_Professora/Coordenadora,cargo_professor_Psicóloga,cargo_professor_Psicólogo,cargo_professor_professora,idade_professor,id_turma,id_disciplina,id_serie,id_periodo,turno_turma,presente,categoria_pais,seq_presenca
168208,2023-04-18,False,True,False,False,False,False,False,6,- Entrada e acolhimento dos alunos \r\n- Calen...,926,0.0,0.0,1.0,False,False,False,False,False,False,True,10.0,16,1.0,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,24.0,512,1,2,6,1,1,Só Pai ou Só Mãe,129
281174,2024-04-25,False,False,False,True,False,False,False,7,Avaliação de língua portuguesa ( P 1).,1822,1.0,1.0,0.0,False,True,False,False,False,False,False,10.0,12,1.0,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,46.0,676,1,8,7,1,1,Só Pai ou Só Mãe,57
52917,2022-03-31,False,False,False,True,False,False,False,6,Revisão das emoções estudadas.,809,0.0,0.0,1.0,False,True,False,False,False,False,False,11.0,14,1.0,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,41.0,148,16,2,5,1,1,Só Pai ou Só Mãe,16
21959,2021-10-05,False,True,False,False,False,False,False,9,P3 - PROVA,397,0.0,0.0,1.0,False,True,False,False,False,False,False,16.0,11,0.0,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,46.0,93,5,15,4,0,1,Só Pai ou Só Mãe,36
307059,2024-06-19,False,False,True,False,False,False,False,3,Aplicação da avaliação P2,219,1.0,0.0,1.0,False,True,False,False,False,False,False,15.0,3,1.0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,39.0,657,1,24,7,1,1,Só Pai ou Só Mãe,404


1. **Ordenação dos Dados**: Primeiro, ordenamos o DataFrame `df` pelas colunas `id_aluno` (identificação do aluno) e `data_aula` (data da aula). Isso garante que as presenças de cada aluno estejam na ordem correta das aulas.

2. **Criando Sequência de Presença**: Utilizamos a função `groupby('id_aluno')` para agrupar os dados por aluno. Em seguida, aplicamos `cumcount()`, que gera uma sequência numérica (começando do 0) dentro de cada grupo. O `+ 1` é adicionado para que a sequência comece em 1.

3. **Resultado**: Isso cria a coluna `'seq_presenca'`, que representa a ordem das presenças para cada aluno dentro do grupo de aulas.

#### <font color='orange'>Criando funções para modelo preditivo de faltas e presenças futuras</font>

Criando um modelo preditivo para probabilidade de faltas e presenças futuras

In [34]:

def criar_abt(df, tamanho_janela=10, passo=1):


    def load_spacy_model():
        # Verifica se o modelo já está instalado
        if not is_package("pt_core_news_sm"):
            # Faz o download do modelo caso não esteja instalado
            from spacy.cli import download
            download("pt_core_news_sm")
        # Carrega o modelo de linguagem
        return spacy.load("pt_core_news_sm")

    # Carregando o modelo de NLP
    # Carrega o modelo de linguagem do spacy
    nlp = load_spacy_model()
    
    # nlp = spacy.load('pt_core_news_sm')

    # Função para criar embeddings do conteúdo ministrado
    def criar_embedding(texto):
        doc = nlp(texto)
        return doc.vector.tolist()

    # Encontra o maior valor da sequência de presenças
    max_seq = df['seq_presenca'].max()
    
    resultados = []

    # Itera sobre as janelas de presenças
    for inicio_janela in range(1, max_seq - tamanho_janela + 2, passo):
        fim_janela = inicio_janela + tamanho_janela - 1


        # Filtra os dados dentro da janela de presenças
        df_janela = (
            df[(df['seq_presenca'] >= inicio_janela) & (df['seq_presenca'] <= fim_janela)]
            .sort_values('seq_presenca')
            .groupby('id_aluno')
            .agg({
                'data_aula': ['first', 'last'],
                'dia_semana_0': 'sum',
                'dia_semana_1': 'sum',
                'dia_semana_2': 'sum',
                'dia_semana_3': 'sum',
                'dia_semana_4': 'sum',
                'dia_semana_5': 'sum',
                'dia_semana_6': 'sum',
                'conteudo_ministrado': lambda x: ' '.join(x),
                'sexo_aluno': 'first',
                'tem_pai': 'first',
                'tem_mae': 'first',
                'raca_aluno_A': 'first',
                'raca_aluno_B': 'first',
                'raca_aluno_I': 'first',
                'raca_aluno_N': 'first',
                'raca_aluno_None': 'first',
                'raca_aluno_P': 'first',
                'raca_aluno_R': 'first',
                'idade_aluno': 'mean',
                'sexo_professor': 'sum',
                'raca_professor_A': 'sum',
                'raca_professor_B': 'sum',
                'raca_professor_N': 'sum',
                'raca_professor_P': 'sum',
                'raca_professor_R': 'sum',
                'cargo_professor_Nenhum': 'sum',
                'cargo_professor_Neuropsicóloga': 'sum',
                'cargo_professor_Professor': 'sum',
                'cargo_professor_Professor Inglês': 'sum',
                'cargo_professor_Professor de Inglês': 'sum',
                'cargo_professor_Professor de Matemática': 'sum',
                'cargo_professor_Professora': 'sum',
                'cargo_professor_Professora de Inglês': 'sum',
                'cargo_professor_Professora de Matemática': 'sum',
                'cargo_professor_Professora/Coordenadora': 'sum',
                'cargo_professor_Psicóloga': 'sum',
                'cargo_professor_Psicólogo': 'sum',
                'idade_professor': 'mean',
                'id_professor': pd.Series.nunique,
                'id_disciplina': pd.Series.nunique,
                'id_serie': pd.Series.nunique,
                'id_turma': pd.Series.nunique,
                'presente': ['sum', 'count'],
                'seq_presenca': lambda x: max(x[x == 1]) if (x == 1).any() else 0
            }).reset_index()
        )

        # Renomeia as colunas do DataFrame após o groupby
        df_janela.columns = [
            'id_aluno', 'first_data_aula', 'last_data_aula', 'sum_dia_semana_1', 'sum_dia_semana_2',
            'sum_dia_semana_3', 'sum_dia_semana_4', 'sum_dia_semana_5', 'sum_dia_semana_6', 'sum_dia_semana_7',
            'conteudo', 'sexo_aluno', 'tem_pai', 'tem_mae', 'raca_aluno_A', 'raca_aluno_B', 'raca_aluno_I',
            'raca_aluno_N', 'raca_aluno_None', 'raca_aluno_P', 'raca_aluno_R', 'idade_aluno_media',
            'sexo_professor_sum', 'raca_professor_A_sum', 'raca_professor_B_sum', 'raca_professor_N_sum',
            'raca_professor_P_sum', 'raca_professor_R_sum', 'cargo_professor_Nenhum_sum',
            'cargo_professor_Neuropsicóloga_sum', 'cargo_professor_Professor_sum',
            'cargo_professor_Professor_Inglês_sum', 'cargo_professor_Professor_de_Inglês_sum',
            'cargo_professor_Professor_de_Matemática_sum', 'cargo_professor_Professora_sum',
            'cargo_professor_Professora_de_Inglês_sum', 'cargo_professor_Professora_de_Matemática_sum',
            'cargo_professor_Professora_Coordenadora_sum', 'cargo_professor_Psicóloga_sum',
            'cargo_professor_Psicólogo_sum', 'idade_professor_media', 'num_professores',
            'num_disciplinas', 'num_series', 'num_turmas', 'num_presencas', 'num_aulas', 'ultima_presenca'
        ]

        # Adiciona a coluna de sequência final da janela
        df_janela['last_seq'] = fim_janela

        # Aplica o embedding ao conteúdo ministrado
        df_janela['conteudo_embedding'] = df_janela['conteudo'].apply(criar_embedding)

        # Busca a presença na próxima aula
        next_presente = df[df['seq_presenca'] == fim_janela + 1][['id_aluno', 'presente']].rename(columns={'presente': 'next_presente'})

        # Junta os dados da próxima presença no DataFrame da janela
        df_janela = pd.merge(df_janela, next_presente, on='id_aluno', how='left')

        # Adiciona os resultados no acumulador
        resultados.append(df_janela)

    # Concatena os resultados de todas as janelas
    return pd.concat(resultados, ignore_index=True)


1. **Objetivo da Função**: A função `criar_abt` constrói uma tabela agregada baseada nas presenças de cada aluno em um intervalo de aulas, criando features (variáveis explicativas) que serão usadas no modelo preditivo.

2. **Parâmetros**:
   - `df`: DataFrame com os dados dos alunos e das aulas.
   - `tamanho_janela`: Define o número de aulas (presenças) que serão usadas para analisar o comportamento de cada aluno.
   - `passo`: Define o número de presenças que o intervalo de análise se deslocará após cada iteração.

3. **Uso do spaCy**: Carrega o modelo de linguagem `spacy` para a língua portuguesa e define a função `criar_embedding`, que transforma o texto de conteúdo ministrado em um vetor numérico (embedding), facilitando a análise de textos.

4. **Laço de Iteração**: O código percorre janelas de tamanho definido pelas presenças de cada aluno, calculando estatísticas e coletando informações de variáveis importantes:
   - **Dados da aula**: Coleta a soma das presenças por dia da semana e concatena o conteúdo ministrado em texto.
   - **Dados do aluno**: Captura informações como sexo, presença dos pais, raça, média de idade e quantidade de presenças.
   - **Dados do professor**: Soma as informações relacionadas a raça, cargos e idade dos professores.
   - **Diversidade de Interações**: Calcula quantos professores, disciplinas, séries e turmas diferentes o aluno teve no período.
   - **Presenças**: Soma quantas presenças o aluno teve na janela e identifica a última aula em que ele esteve presente.

5. **Cálculo do Target**: Após gerar as variáveis de cada janela, o código também busca a presença na próxima aula (`next_presente`), que será o target para o modelo preditivo.

6. **Resultado**: Ao final, todos os dados gerados para cada janela são concatenados e retornados como um novo DataFrame.


In [35]:
# Chama a função criar_abt para gerar a tabela agregada
abt = criar_abt(df_prox)

# Filtra as linhas onde 'next_presente' não é nulo
abt = abt[abt['next_presente'].notnull()]

# Ordena o DataFrame pela coluna 'first_data_aula'
abt = abt.sort_values('first_data_aula').reset_index(drop=True)



1. **`criar_abt(df_prox)`**: A função `criar_abt` é chamada para criar a tabela agregada (ABT).
2. **`abt['next_presente'].notnull()`**: Usa o método `notnull()` do `pandas` para filtrar as linhas onde a coluna `next_presente` não é nula.
3. **`sort_values('first_data_aula')`**: Ordena o DataFrame pela coluna `'first_data_aula'` para garantir que os dados estejam em ordem cronológica. O método `reset_index(drop=True)` remove a antiga numeração de índices após a ordenação.

In [36]:
# Remove as colunas 'conteudo' e 'conteudo_embedding'
abt_filtered = abt.drop(columns=['conteudo', 'conteudo_embedding'])

# Expande a lista de embeddings (conteúdo_embedding) em múltiplas colunas
conteudo_embedding_expanded = pd.DataFrame(abt['conteudo_embedding'].tolist(), index=abt.index)

# Concatena o DataFrame original com as novas colunas de embedding
abt_final = pd.concat([abt_filtered, conteudo_embedding_expanded], axis=1)

# Exibe o resultado final
abt_final


,id_aluno,first_data_aula,last_data_aula,sum_dia_semana_1,sum_dia_semana_2,sum_dia_semana_3,sum_dia_semana_4,sum_dia_semana_5,sum_dia_semana_6,sum_dia_semana_7,sexo_aluno,tem_pai,tem_mae,raca_aluno_A,raca_aluno_B,raca_aluno_I,raca_aluno_N,raca_aluno_None,raca_aluno_P,raca_aluno_R,idade_aluno_media,sexo_professor_sum,raca_professor_A_sum,raca_professor_B_sum,raca_professor_N_sum,raca_professor_P_sum,raca_professor_R_sum,cargo_professor_Nenhum_sum,cargo_professor_Neuropsicóloga_sum,cargo_professor_Professor_sum,cargo_professor_Professor_Inglês_sum,cargo_professor_Professor_de_Inglês_sum,cargo_professor_Professor_de_Matemática_sum,cargo_professor_Professora_sum,cargo_professor_Professora_de_Inglês_sum,cargo_professor_Professora_de_Matemática_sum,cargo_professor_Professora_Coordenadora_sum,cargo_professor_Psicóloga_sum,cargo_professor_Psicólogo_sum,idade_professor_media,num_professores,num_disciplinas,num_series,num_turmas,num_presencas,num_aulas,ultima_presenca,last_seq,next_presente,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
0,648,2021-07-01,2021-08-17,1,4,0,5,0,0,0,1.0,0.0,1.0,False,False,False,False,False,True,False,10.0,10.0,0,6,0,0,4,0,0,0,0,0,0,10,0,0,0,0,0,46.4,2,4,2,2,9,10,1,10,0.0,0.464691,-1.198423,-0.319709,0.602826,-0.460871,1.102701,0.704168,0.853496,-0.009229,0.622204,0.452460,-2.836177,-1.992858,1.045671,-1.642192,1.548579,0.522448,0.780726,1.101357,0.818639,-0.097104,0.239930,0.080852,0.198630,-0.890790,-0.782643,1.410378,0.247345,1.862815,0.916794,-0.138691,1.315967,0.865970,0.294602,0.399981,-0.821849,-0.432646,0.700167,-0.233130,0.618022,-1.726568,2.311276,-0.858766,1.622699,0.139805,0.816618,-0.911285,1.377313,-1.339101,-1.049585,-0.658509,0.292452,1.949671,0.319016,0.976979,-0.751922,0.579159,-0.586001,-1.081194,-1.101541,-0.748312,-1.767539,-1.438028,-1.489322,-0.372536,0.985170,-0.870256,-1.297789,-1.058778,0.659834,-1.343446,1.526212,1.048456,-1.556131,-0.236190,-1.028931,-0.554941,0.386188,2.106617,-1.995267,-0.088607,1.010175,-1.295087,1.418227,0.266865,0.647785,0.606404,0.413301,-1.151923,1.166774,0.870474,-1.508583,1.903979,1.765981,-1.539079,0.019102
1,481,2021-07-01,2021-08-17,4,3,2,1,0,0,0,1.0,0.0,1.0,False,True,False,False,False,False,False,14.0,10.0,0,3,0,0,7,0,0,0,0,0,0,10,0,0,0,0,0,43.8,3,3,1,1,10,10,1,10,1.0,-0.073422,-0.711597,0.341229,0.716728,0.310992,0.983920,0.866293,1.014369,0.307688,-0.136745,0.239743,-2.273412,-1.965942,1.003965,-1.419866,1.729892,0.465460,0.265484,0.560922,0.985313,1.118845,0.106258,-0.788691,1.096229,-1.335220,-0.296204,1.267547,0.519375,2.182246,0.705323,0.741574,0.965964,0.875002,-0.318132,1.435704,-0.588161,-0.514121,0.081712,-0.742954,-0.373873,-1.616383,1.426424,-0.651520,1.620814,0.624212,0.122211,-1.464309,0.939843,-1.150770,-1.419779,-1.389726,-0.188227,1.731806,0.485782,0.163470,0.112734,0.025611,-0.745826,-1.255155,-1.278717,-0.754777,-0.651368,-1.434665,-1.241882,0.646737,0.890642,-1.073662,-1.112513,-0.248149,0.930719,-1.112172,1.277486,1.265674,-1.521356,-0.027169,0.071664,-0.988226,0.122071,1.073685,-1.859833,0.132486,1.305501,-1.279922,1.328429,0.520989,1.093549,0.686937,0.732217,-1.404893,1.191996,0.696156,-1.721939,1.838985,1.061773,-1.971193,-0.030658
2,482,2021-07-01,2021-08-17,4,3,2,1,0,0,0,0.0,0.0,1.0,False,True,False,False,False,False,False,13.0,10.0,0,3,0,0,7,0,0,0,0,0,0,10,0,0,0,0,0,43.8,3,3,1,1,9,10,1,10,1.0,-0.073422,-0.711597,0.341229,0.716728,0.310992,0.983920,0.866293,1.014369,0.307688,-0.136745,0.239743,-2.273412,-1.965942,1.003965,-1.419866,1.729892,0.465460,0.265484,0.560922,0.985313,1.118845,0.106258,-0.788691,1.096229,-1.335220,-0.296204,1.267547,0.519375,2.182246,0.705323,0.741574,0.965964,0.875002,-0.318132,1.435704,-0.588161,-0.514121,0.081712,-0.742954,-0.373873,-1.616383,1.426424,-0.651520,1.620814,

1. **`drop()`**: Remove as colunas `'conteudo'` e `'conteudo_embedding'` do DataFrame.
2. **Expansão do `conteudo_embedding`**: Usamos `pd.DataFrame(abt['conteudo_embedding'].tolist())` para transformar a lista de embeddings em várias colunas, uma para cada valor na lista.
3. **Concatenação**: Unimos o DataFrame original sem as colunas removidas com o DataFrame expandido dos embeddings usando `pd.concat()`.


In [37]:
pickle.dump(abt_final, open("abt.pkl", "wb"))

Este código salva o DataFrame `abt` em um arquivo chamado `"abt.pkl"`, permitindo que a recuperação seja de forma rapida, mais tarde caso necessário podemos recuperar sem precisar refazer todo o processo de cálculo.

O código salva o DataFrame `abt` em um arquivo usando a biblioteca `pickle`:

1. **`pickle.dump`**: Essa função serializa (converte em formato binário) o objeto `abt` para que ele possa ser salvo em um arquivo e carregado posteriormente.
   
2. **`open("abt.pkl", "wb")`**: Abre um arquivo chamado `abt.pkl` em modo de escrita binária (`wb`), onde o DataFrame serializado será salvo.

3. **Finalidade**: O arquivo `"abt.pkl"` pode ser reutilizado em outro momento, carregando os dados sem precisar recriar ou recalcular o DataFrame.



In [2]:
# Caminho do arquivo .pkl
file_path = 'abt.pkl'

# Abrir o arquivo .pkl e carregar os dados
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Se os dados forem compatíveis com um DataFrame, basta convertê-los
abt = pd.DataFrame(data)

# Verificar o DataFrame
abt

,id_aluno,first_data_aula,last_data_aula,sum_dia_semana_1,sum_dia_semana_2,sum_dia_semana_3,sum_dia_semana_4,sum_dia_semana_5,sum_dia_semana_6,sum_dia_semana_7,sexo_aluno,tem_pai,tem_mae,raca_aluno_A,raca_aluno_B,raca_aluno_I,raca_aluno_N,raca_aluno_None,raca_aluno_P,raca_aluno_R,idade_aluno_media,sexo_professor_sum,raca_professor_A_sum,raca_professor_B_sum,raca_professor_N_sum,raca_professor_P_sum,raca_professor_R_sum,cargo_professor_Nenhum_sum,cargo_professor_Neuropsicóloga_sum,cargo_professor_Professor_sum,cargo_professor_Professor_Inglês_sum,cargo_professor_Professor_de_Inglês_sum,cargo_professor_Professor_de_Matemática_sum,cargo_professor_Professora_sum,cargo_professor_Professora_de_Inglês_sum,cargo_professor_Professora_de_Matemática_sum,cargo_professor_Professora_Coordenadora_sum,cargo_professor_Psicóloga_sum,cargo_professor_Psicólogo_sum,idade_professor_media,num_professores,num_disciplinas,num_series,num_turmas,num_presencas,num_aulas,ultima_presenca,last_seq,next_presente,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
0,648,2021-07-01,2021-08-17,1,4,0,5,0,0,0,1.0,0.0,1.0,False,False,False,False,False,True,False,10.0,10.0,0,6,0,0,4,0,0,0,0,0,0,10,0,0,0,0,0,46.4,2,4,2,2,9,10,1,10,0.0,0.464691,-1.198423,-0.319709,0.602826,-0.460871,1.102701,0.704168,0.853496,-0.009229,0.622204,0.452460,-2.836177,-1.992858,1.045671,-1.642192,1.548579,0.522448,0.780726,1.101357,0.818639,-0.097104,0.239930,0.080852,0.198630,-0.890790,-0.782643,1.410378,0.247345,1.862815,0.916794,-0.138691,1.315967,0.865970,0.294602,0.399981,-0.821849,-0.432646,0.700167,-0.233130,0.618022,-1.726568,2.311276,-0.858766,1.622699,0.139805,0.816618,-0.911285,1.377313,-1.339101,-1.049585,-0.658509,0.292452,1.949671,0.319016,0.976979,-0.751922,0.579159,-0.586001,-1.081194,-1.101541,-0.748312,-1.767539,-1.438028,-1.489322,-0.372536,0.985170,-0.870256,-1.297789,-1.058778,0.659834,-1.343446,1.526212,1.048456,-1.556131,-0.236190,-1.028931,-0.554941,0.386188,2.106617,-1.995267,-0.088607,1.010175,-1.295087,1.418227,0.266865,0.647785,0.606404,0.413301,-1.151923,1.166774,0.870474,-1.508583,1.903979,1.765981,-1.539079,0.019102
1,481,2021-07-01,2021-08-17,4,3,2,1,0,0,0,1.0,0.0,1.0,False,True,False,False,False,False,False,14.0,10.0,0,3,0,0,7,0,0,0,0,0,0,10,0,0,0,0,0,43.8,3,3,1,1,10,10,1,10,1.0,-0.073422,-0.711597,0.341229,0.716728,0.310992,0.983920,0.866293,1.014369,0.307688,-0.136745,0.239743,-2.273412,-1.965942,1.003965,-1.419866,1.729892,0.465460,0.265484,0.560922,0.985313,1.118845,0.106258,-0.788691,1.096229,-1.335220,-0.296204,1.267547,0.519375,2.182246,0.705323,0.741574,0.965964,0.875002,-0.318132,1.435704,-0.588161,-0.514121,0.081712,-0.742954,-0.373873,-1.616383,1.426424,-0.651520,1.620814,0.624212,0.122211,-1.464309,0.939843,-1.150770,-1.419779,-1.389726,-0.188227,1.731806,0.485782,0.163470,0.112734,0.025611,-0.745826,-1.255155,-1.278717,-0.754777,-0.651368,-1.434665,-1.241882,0.646737,0.890642,-1.073662,-1.112513,-0.248149,0.930719,-1.112172,1.277486,1.265674,-1.521356,-0.027169,0.071664,-0.988226,0.122071,1.073685,-1.859833,0.132486,1.305501,-1.279922,1.328429,0.520989,1.093549,0.686937,0.732217,-1.404893,1.191996,0.696156,-1.721939,1.838985,1.061773,-1.971193,-0.030658
2,482,2021-07-01,2021-08-17,4,3,2,1,0,0,0,0.0,0.0,1.0,False,True,False,False,False,False,False,13.0,10.0,0,3,0,0,7,0,0,0,0,0,0,10,0,0,0,0,0,43.8,3,3,1,1,9,10,1,10,1.0,-0.073422,-0.711597,0.341229,0.716728,0.310992,0.983920,0.866293,1.014369,0.307688,-0.136745,0.239743,-2.273412,-1.965942,1.003965,-1.419866,1.729892,0.465460,0.265484,0.560922,0.985313,1.118845,0.106258,-0.788691,1.096229,-1.335220,-0.296204,1.267547,0.519375,2.182246,0.705323,0.741574,0.965964,0.875002,-0.318132,1.435704,-0.588161,-0.514121,0.081712,-0.742954,-0.373873,-1.616383,1.426424,-0.651520,1.620814,

In [4]:
set(abt.columns)

{0,
 1,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 2,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 3,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 4,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 5,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 6,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 7,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 8,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 9,
 90,
 91,
 92,
 93,
 94,
 95,
 'cargo_professor_Nenhum_sum',
 'cargo_professor_Neuropsicóloga_sum',
 'cargo_professor_Professor_Inglês_sum',
 'cargo_professor_Professor_de_Inglês_sum',
 'cargo_professor_Professor_de_Matemática_sum',
 'cargo_professor_Professor_sum',
 'cargo_professor_Professora_Coordenadora_sum',
 'cargo_professor_Professora_de_Inglês_sum',
 'cargo_professor_Professora_de_Matemática_sum',
 'cargo_professor_Professora_sum',
 'cargo_professor_Psicóloga_sum',
 'cargo_professor_Psicólogo_sum',
 'first_data_aula',
 'id_aluno',
 'idade_aluno_m

In [8]:
len(abt['id_aluno'].unique())

2109

In [ ]:
abt[]

In [10]:
abt[['idade_aluno_media']].describe().T

,count,mean,std,min,25%,50%,75%,max
idade_aluno_media,291748.0,13.917799,2.96587,7.0,12.0,14.0,16.0,36.0


In [11]:
abt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291928 entries, 0 to 291927
Columns: 145 entries, id_aluno to 95
dtypes: datetime64[ns](2), float64(103), int64(16), object(24)
memory usage: 322.9+ MB


#### <font color='orange'>Geração da Modelagem - Separando Bases</font>

Essa divisão é essencial para garantir que o modelo seja treinado e avaliado de forma adequada, sem sobreposição dos dados de treino e teste.

In [73]:
# Separando o conjunto de treino (datas antes de 01/01/2024)
abt_train = abt[abt['first_data_aula'] < datetime(2024, 1, 1)]

# Separando o conjunto de teste (datas após 01/01/2024)
abt_test = abt[abt['first_data_aula'] > datetime(2024, 1, 1)]


1. **Objetivo**: Estamos separando os dados em duas partes: uma para treinamento do modelo e outra para teste, com base nas datas.

2. **`abt_train`**: Contém as linhas onde a coluna `first_data_aula` (primeira aula) tem uma data **antes de 1º de janeiro de 2024**. Esses dados serão usados para treinar o modelo.

3. **`abt_test`**: Contém as linhas onde `first_data_aula` é **após 1º de janeiro de 2024**. Esses dados serão usados para testar a performance do modelo.



**Importância**: Ao separar as variáveis explicativas (X) das variáveis target (y), estamos preparando os dados para que possam ser usados no treinamento de um modelo preditivo, com o objetivo de prever se o aluno estará presente ou ausente na próxima aula.

In [74]:
# Separando as features (X) e o target (y) para o conjunto de treino
X_train = abt_train.drop(columns=['id_aluno', 'next_presente', 'first_data_aula', 'last_data_aula', 'last_seq'])
y_train = abt_train['next_presente']

# Separando as features (X) e o target (y) para o conjunto de teste
X_test = abt_test.drop(columns=['id_aluno', 'next_presente', 'first_data_aula', 'last_data_aula', 'last_seq'])
y_test = abt_test['next_presente']


In [75]:
X_train.shape, X_test.shape

((242702, 140), (49226, 140))

In [76]:
X_train.shape, X_test.shape

((242702, 140), (49226, 140))

**Separação das Features (X) e Target (y)**:
   - **X_train** e **X_test**: São os conjuntos de dados de treino e teste, contendo todas as variáveis explicativas (features) menos as colunas que não queremos usar no modelo, como `id_aluno`, `next_presente` (que é o target), `first_data_aula`, `last_data_aula`, e `last_seq`.
   
   - **y_train** e **y_test**: São as variáveis target, ou seja, o que o modelo vai tentar prever. Nesse caso, é a coluna `next_presente`, que indica se o aluno estava presente na próxima aula.

In [77]:
# Calcular as contagens de cada valor no y_train
value_counts = y_train.value_counts()

# Calcular a soma total de ocorrências
total_count = value_counts.sum()

# Normalizar as contagens (proporções)
value_counts_normalized = value_counts / total_count

# Exibir o resultado
value_counts_normalized


next_presente
1.0    0.76851
0.0    0.23149
Name: count, dtype: float64


1. **`value_counts()`**: Conta quantas vezes cada valor aparece no `y_train`. Neste caso, contamos as ocorrências do target `next_presente` (presenças ou ausências).
   
2. **`sum()`**: Calcula o total de ocorrências para que possamos normalizar os valores, ou seja, converter as contagens em proporções.

3. **Normalização**: Dividimos as contagens pelo total para obter as proporções, o que facilita a interpretação da distribuição dos dados.

4. **Resultado**: `value_counts_normalized` exibe a proporção de cada classe (presente ou ausente) no conjunto de treino, ajudando a entender o balanceamento dos dados.

In [78]:
# Converter todos os nomes de colunas para strings
X_train.columns = X_train.columns.astype(str)

# Aplicar RandomUnderSampler para balancear as classes
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)


1. **`RandomUnderSampler`**: Esse método reduz o número de amostras da classe majoritária no conjunto de treino para equilibrar o número de amostras entre as classes (por exemplo, presença e ausência).
   
2. **`fit_resample()`**: Aplica o processo de undersampling, criando novos conjuntos `X_resampled` e `y_resampled`, onde as classes estão balanceadas.


In [79]:
y_resampled.value_counts(normalize=True)

next_presente
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64

In [80]:
print(f"{y_resampled.shape=}")

X_resampled = X_resampled.fillna(0)

print(X_resampled[X_resampled.isna().any(axis=1)])

print(y_resampled[y_resampled.isna()])

y_resampled.shape=(112366,)
Empty DataFrame
Columns: [sum_dia_semana_1, sum_dia_semana_2, sum_dia_semana_3, sum_dia_semana_4, sum_dia_semana_5, sum_dia_semana_6, sum_dia_semana_7, sexo_aluno, tem_pai, tem_mae, raca_aluno_A, raca_aluno_B, raca_aluno_I, raca_aluno_N, raca_aluno_None, raca_aluno_P, raca_aluno_R, idade_aluno_media, sexo_professor_sum, raca_professor_A_sum, raca_professor_B_sum, raca_professor_N_sum, raca_professor_P_sum, raca_professor_R_sum, cargo_professor_Nenhum_sum, cargo_professor_Neuropsicóloga_sum, cargo_professor_Professor_sum, cargo_professor_Professor_Inglês_sum, cargo_professor_Professor_de_Inglês_sum, cargo_professor_Professor_de_Matemática_sum, cargo_professor_Professora_sum, cargo_professor_Professora_de_Inglês_sum, cargo_professor_Professora_de_Matemática_sum, cargo_professor_Professora_Coordenadora_sum, cargo_professor_Psicóloga_sum, cargo_professor_Psicólogo_sum, idade_professor_media, num_professores, num_disciplinas, num_series, num_turmas, num_presencas

#### <font color='orange'>Modelagem - Treinando Modelos</font>

Este código busca encontrar os melhores hiperparâmetros para o modelo de `RandomForestClassifier` utilizando a busca aleatória (`RandomizedSearchCV`). O objetivo é testar várias combinações de hiperparâmetros e identificar aquela que oferece o melhor desempenho, garantindo que o modelo seja o mais otimizado possível.

1. **Definição dos Hiperparâmetros**:
   - O dicionário `param_dist` define os possíveis valores para vários hiperparâmetros do `RandomForestClassifier`. Aqui estão os parâmetros sendo otimizados:
     - **`n_estimators`**: Número de árvores na floresta (100, 200, 300, ou 600).
     - **`max_depth`**: Profundidade máxima de cada árvore (5 ou 10).
     - **`min_samples_split`**: Número mínimo de amostras necessárias para dividir um nó (2, 5, ou 10).
     - **`min_samples_leaf`**: Número mínimo de amostras necessárias em cada folha (1, 2, ou 4).


2. **RandomizedSearchCV**:
   - Utilizamos `RandomizedSearchCV` para realizar a busca dos melhores hiperparâmetros. Isso testa combinações aleatórias dos hiperparâmetros especificados em `param_dist`, em vez de todas as combinações possíveis (como seria o caso com `GridSearchCV`).
   
   - **Parâmetros principais**:
     - **`estimator`**: Um classificador de floresta aleatória (`RandomForestClassifier`).
     - **`param_distributions`**: O conjunto de hiperparâmetros a serem testados.
     - **`n_iter=20`**: Testa 20 combinações diferentes de hiperparâmetros.
     - **`cv=5`**: Realiza validação cruzada com 5 divisões dos dados (5-fold cross-validation).
     - **`n_jobs=-1`**: Utiliza todos os processadores disponíveis para acelerar o processo.
     - **`random_state=42`**: Garante reprodutibilidade dos resultados.
     - **`verbose=2`**: Mostra o progresso do processo no console.

In [81]:
param_dist = {
   'n_estimators': [100, 200, 300, 600],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_cv = RandomizedSearchCV(
    estimator= RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

rf_cv.fit(X_resampled, y_resampled)
best_rf = rf_cv.best_estimator_

Fitting 5 folds for each of 20 candidates, totalling 100 fits


3. **Treinamento do Modelo**:
   - O método `fit(X_resampled, y_resampled)` treina o modelo com os dados balanceados e procura os melhores hiperparâmetros entre as combinações testadas.

4. **Melhor Modelo**:
   - **`best_rf = rf_cv.best_estimator_`**: Após a busca, `best_estimator_` contém a melhor combinação de hiperparâmetros encontrada durante a pesquisa. Esse será o modelo de floresta aleatória com o melhor desempenho.



In [82]:
# Versão de melhor modelo dos hiperparametros
rf_cv.best_params_

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_depth': 5}

In [63]:
y_train.value_counts()

next_presente
1.0    186519
0.0     56183
Name: count, dtype: int64

Este código cria e treina um modelo de **RandomForestClassifier** com parâmetros específicos que controlam o comportamento das árvores no modelo. Os hiperparâmetros, como a profundidade máxima da árvore e o número mínimo de amostras em cada divisão ou folha, foram configurados manualmente para tentar garantir um bom desempenho do modelo.

1. **Criação do Modelo**:
   - Um modelo de **`RandomForestClassifier`** é criado usando hiperparâmetros específicos, diretamente passados como um dicionário (`**{}`) com os valores que foram previamente otimizados ou escolhidos:
     - **`n_estimators=200`**: O modelo usa 200 árvores de decisão.
     - **`min_samples_split=5`**: Um nó interno precisa de pelo menos 5 amostras para ser dividido.
     - **`min_samples_leaf=1`**: Cada folha (nó terminal) precisa ter no mínimo 1 amostras.
     - **`max_depth=5`**: A profundidade máxima de cada árvore é limitada a 5 níveis.
     - **`random_state=42`**: Garante que os resultados sejam reprodutíveis.

In [84]:
best_rf = RandomForestClassifier(**{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_depth': 5,
 'random_state': 42}).fit(X_resampled, y_resampled)

2. **Treinamento do Modelo**:
   - **`.fit(X_resampled, y_resampled)`**: O modelo é treinado com os dados balanceados de treino. O **`X_resampled`** contém as variáveis explicativas (features), e **`y_resampled`** contém a variável alvo (target) que indica a presença ou ausência do aluno na próxima aula.

<font color='orange'>**Relatório de classificação**</font>

Esse código usa o modelo treinado (**`best_rf`**) para prever os resultados no conjunto de teste (**`X_test`**) e, em seguida, avalia a performance do modelo comparando as previsões com os valores reais (**`y_test`**). O **`classification_report`** oferece uma visão clara da precisão, revocação, F1-score, e suporte, ajudando a entender quão bem o modelo está generalizando para novos dados.

1. **Previsão dos Dados de Teste**:
   - **`y_pred = best_rf.predict(X_test)`**: O modelo treinado (**`best_rf`**) é utilizado para fazer previsões nos dados de teste (**`X_test`**). O resultado, **`y_pred`**, contém as previsões da presença ou ausência do aluno para cada observação no conjunto de teste.


In [85]:
# Converter todos os nomes de colunas para strings
X_test.columns = X_test.columns.astype(str)

y_pred = best_rf.predict(X_test)
    
# Gerar e imprimir o relatório de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))


Relatório de Classificação:
              precision    recall  f1-score   support

         0.0       0.44      0.24      0.31      7063
         1.0       0.88      0.95      0.91     42163

    accuracy                           0.85     49226
   macro avg       0.66      0.60      0.61     49226
weighted avg       0.82      0.85      0.83     49226



2. **Relatório de Classificação**:
   - **`classification_report(y_test, y_pred)`**: Essa função gera um relatório que compara as previsões (**`y_pred`**) com os valores reais (**`y_test`**). O relatório inclui as seguintes métricas de avaliação:
     - **Precisão (Precision)**: A proporção de verdadeiros positivos sobre o total de positivos previstos (quantas das previsões positivas estavam corretas).
     - **Revocação (Recall)**: A proporção de verdadeiros positivos sobre o total de positivos reais (quantos dos casos positivos reais foram detectados pelo modelo).
     - **F1-Score**: A média harmônica entre precisão e revocação, indicando o equilíbrio entre essas duas métricas.
     - **Suporte (Support)**: O número de ocorrências reais de cada classe no conjunto de dados.

3. **Impressão do Relatório**:
   - O relatório de classificação é impresso no console, fornecendo uma análise detalhada de como o modelo performou em prever as presenças ou ausências dos alunos no conjunto de teste.

<font color='orange'>**Análise do Resultado de Classificação:**</font>

1. **Classes Analisadas**:
   - **Classe 0.0**: Representa a ausência do aluno.
   - **Classe 1.0**: Representa a presença do aluno.

2. **Precisão (Precision)**:
   - A **precisão** para a **classe 0.0** (ausência) é **0.44**, o que significa que, entre todas as vezes que o modelo previu que um aluno estaria ausente, apenas **44%** dessas previsões estavam corretas.
   - Para a **classe 1.0** (presença), a precisão é muito alta, **0.88**, indicando que, quando o modelo previu que um aluno estaria presente, **88%** das previsões estavam corretas.

3. **Revocação (Recall)**:
   - Para a **classe 0.0**, a revocação é **0.24**, o que mostra que o modelo teve dificuldades em identificar corretamente as ausências dos alunos, acertando apenas **24%** das vezes que o aluno realmente esteve ausente.
   - Já para a **classe 1.0**, a revocação é **0.95**, o que significa que o modelo identificou corretamente **95%** das presenças.

4. **F1-Score**:
   - O **f1-score** para a **classe 0.0** é **0.31**, indicando um desempenho fraco em prever ausências.
   - Para a **classe 1.0**, o **f1-score** é **0.91**, refletindo um bom equilíbrio entre precisão e revocação para prever presenças.

5. **Acurácia (Accuracy)**:
   - A **acurácia geral** do modelo é **0.85**, o que significa que o modelo está correto **85%** das vezes. Porém, essa métrica está sendo influenciada pelo fato de que há muito mais dados de presenças do que de ausências (desequilíbrio de classes).

6. **Médias**:
   - O **macro average** (média das métricas de ambas as classes) mostra que o desempenho global em termos de precisão, revocação e f1-score é bem mais baixo para a classe 0.0 (ausência) do que para a classe 1.0 (presença), indicando que o modelo tem dificuldades em lidar com as ausências.
   - O **weighted average** pondera essas métricas de acordo com o número de instâncias em cada classe, resultando em um desempenho geral mais alto, especialmente porque a classe 1.0 (presença) domina o conjunto de dados.


<font color='orange'>**Feature importance**</font>

Este código identifica as variáveis que mais impactam as previsões do modelo **RandomForestClassifier**. Ele exibe as importâncias das features em ordem decrescente, permitindo que você visualize as 25 variáveis que o modelo considera mais relevantes para prever a presença ou ausência dos alunos. Isso ajuda a entender quais fatores estão influenciando as decisões do modelo.

1. **Importância das Features**:
   - **`best_rf.feature_importances_`**: Esse atributo do `RandomForestClassifier` retorna uma lista de valores que indicam a importância de cada feature (variável explicativa) no modelo. Quanto maior o valor, mais relevante é a feature para a previsão do modelo.

2. **Criação de DataFrame**:
   - **`pd.DataFrame()`**: Um DataFrame é criado para organizar e exibir as importâncias das features. 
     - As importâncias das features são passadas como a primeira coluna.
     - O índice é definido como os nomes das colunas de **`X_train.columns`**, ou seja, os nomes das variáveis explicativas.
     - A coluna se chama **`'importance'`** para deixar claro que ela representa as importâncias das variáveis.


In [86]:
# Feature importance
feature_importance = pd.DataFrame(
    best_rf.feature_importances_,
    index=X_train.columns,
    columns=['importance']
).sort_values('importance', ascending=False)

feature_importance.head(25)

,importance
num_presencas,0.501081
idade_aluno_media,0.047648
cargo_professor_Nenhum_sum,0.042959
tem_pai,0.038962
cargo_professor_Psicóloga_sum,0.028642
num_disciplinas,0.022466
cargo_professor_Professor_sum,0.020005
tem_mae,0.016313
idade_professor_media,0.014412
num_professores,0.011950


3. **Ordenação das Features**:
   - **`sort_values('importance', ascending=False)`**: O DataFrame é ordenado de forma decrescente, mostrando as features mais importantes primeiro.

4. **Exibição das 25 Principais Features**:
   - **`head(25)`**: Exibe as 25 variáveis mais importantes para o modelo.

<font color='orange'>**Armazena o modelo treinado**</font>

Esse código cria um dicionário chamado **`dict_model`** que armazena o modelo treinado (**`best_rf`**) e as features usadas no treinamento (**`X_train.columns`**). Esse dicionário facilita o acesso ao modelo e às colunas originais, sendo útil para salvar ou compartilhar o modelo, garantindo que ele possa ser reutilizado corretamente com dados futuros.

1. **Criação de um Dicionário**:
   - **`dict_model`**: Um dicionário é criado para armazenar informações importantes sobre o modelo treinado.


In [87]:
dict_model = {
    'model': best_rf,
    'cols': X_train.columns
}

2. **Chaves e Valores**:
   - **`'model': best_rf`**: Armazena o modelo treinado **`best_rf`** no dicionário sob a chave `'model'`. Isso permite que você acesse o modelo facilmente no futuro.
   - **`'cols': X_train.columns`**: Armazena os nomes das colunas (features) que foram usadas no treinamento, extraídas de **`X_train.columns`**. Isso é útil para garantir que o modelo seja aplicado corretamente a novos dados que tenham o mesmo conjunto de colunas.

Esse código salva o dicionário **`dict_model`** (que contém o modelo treinado e as colunas) no arquivo **`model2.pkl`**, permitindo que você carregue o modelo posteriormente para reutilização.

In [88]:
pickle.dump(dict_model, open('./documents/model/model2.pkl', 'wb'))

Este código salva o dicionário que contém o modelo treinado e as colunas usadas no treinamento em um arquivo.

1. **`pickle.dump`**: Converte o dicionário **`dict_model`** para um formato binário e o salva em um arquivo.
   
2. **`open('../documents/model/model2.pkl', 'wb')`**: Abre ou cria um arquivo chamado **`model2.pkl`** no diretório especificado, em modo de escrita binária (`'wb'`), para armazenar o modelo.


#### <font color='orange'>Explorar Quantidade de Aulas no Geral</font>

In [122]:
# Agrupando por 'data_aula' e contando a frequência
data_aulas = df.groupby('data_aula').size().reset_index(name='count').sort_values('data_aula')

# Criando o gráfico de barras com Plotly Express
fig = px.bar(
    data_aulas,
    x='data_aula',
    y='count',
    title="Número de Aulas por Data",
    labels={'data_aula': 'Data da Aula', 'count': 'Número de Aulas'},
    width=900,
    height=500
)

# Exibir o gráfico
fig.show()
